# Data preparation

In [1]:
# Separe aqui seus dados pra fazer a inferencia

# Setup Llama 3.1:8b Instruct

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import os
import csv
from pathlib import Path
import json
import pandas as pd
from tqdm.auto import tqdm

In [3]:
import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='llm_inferences.log',
    filemode='a'
)

In [4]:
local_path = "/home/jovyan/datalake/models/llama3-8b"

print(f"Loading tokenizer from {local_path}...")
tokenizer = AutoTokenizer.from_pretrained(
    local_path,
    local_files_only=True,
    trust_remote_code=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("pad_token defined as eos_token for tokenizer.")

print(f"Loading model from {local_path}...")

model = AutoModelForCausalLM.from_pretrained(
    local_path,
    local_files_only=True,
    trust_remote_code=True,
    dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="cuda"
)

Loading tokenizer from /home/jovyan/datalake/models/llama3-8b...
pad_token defined as eos_token for tokenizer.
Loading model from /home/jovyan/datalake/models/llama3-8b...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

Device set to use cuda


In [6]:
generation_params = {
    "max_new_tokens": 256,
    "do_sample": False,
    #"temperature": 0.7,
    "truncation": False,
    "return_full_text": False,
}
batch_size = 64

In [7]:
def save_batch_to_csv(batch, results_to_save):
    """
    Save a batch result into a CSV file, creating the header if the file
    does not exist or is empty.

    Args:
        batch (int): Batch ID
        results_to_save (list): List of llm results obtained (prompt + output)
    """

    output_csv_file = f"llm_inferences/llama3.1-8b-instruct/{batch}.csv"
    fieldnames = ["original_id", "original_prompt", "llm_response"]

    output_dir = Path(output_csv_file).parent
    output_dir.mkdir(parents=True, exist_ok=True)

    write_header = not os.path.exists(output_csv_file) or os.path.getsize(output_csv_file) == 0

    try:
        with open(output_csv_file, 'a', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            if write_header:
                writer.writeheader()
                logging.info(f"Header written to '{output_csv_file}'.")

            writer.writerows(results_to_save)

    except IOError as e:
        logging.error(f"Error on saving batch to CSV '{output_csv_file}': {e}")

In [8]:
PROGRESS_FILE = 'inferences.json'

def load_progress():
    """Load the progress from a JSON file."""
    if os.path.exists(PROGRESS_FILE):
        with open(PROGRESS_FILE, 'r') as f:
            logging.info(f"Progress file '{PROGRESS_FILE}' found. Loading state.")
            return json.load(f)
    logging.info(f"No progress file '{PROGRESS_FILE}' found. Starting from zero.")
    return {}

def save_progress(progress_data):
    """Save the current progress into a JSON file"""
    with open(PROGRESS_FILE, 'w') as f:
        json.dump(progress_data, f, indent=4)

In [9]:
progress = load_progress()

In [10]:
ALL_FEW_SHOT_SAMPLES = {
    'fold_1': {
        'toxicidade': {
            'amostra_1': [
                {'role': 'user', 'content': "Text: 'Pra Onde Olhar Na Hora do GARGA #shorts     #hackeandoamatrix #morenotatuado #homemdealtovalor-----------------------------------------------------------------------------------------------------------Esse é um canal de opinião, onde analiso dinâmicas sociais entre homens e mulheres do ponto de vista de um homem.Meu compromisso é dar minha contribuição sincera e respeitosa sobre assuntos da atualidade que interessam ao homem de alto valor.Quando o homem começa a seguir o próprio caminho aprende que precisa se manter bem informado.💊DEIXE O LIKE!                       💊SE INSCREVA!             👉Compartilhe esse vídeo com alguém que vc gosta:'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'MULHER DIZ QUE NAMORARIA UM HOMEM QUE GANHA SALÁRIO MÍNIMO! CONFIA MIQUEINHA #memes #humor #comedia seja bem vindo a mais um vídeo do canal Não se esqueça inscrever no CANAL e deixar o like  além disso ative o sininho de notificações para não perder nenhum vídeo.MULHER DIZ QUE NAMORARIA UM HOMEM QUE GANHA SALÁRIO MÍNIMO! CONFIA MIQUEINHA'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'MULHER NÃO PODE VER O HOMEM EM PAZ! VAI CASAR MIQUEINHA QUE VAI DÁ BOM!  #amelhorredpillvemdelas Olá! seja bem vindo a mais um video do canal, hoje vamos falar, sobre o porquê que o homem deve malhar e treinar. não se esqueça de se inscrever no canal e deixar o like  além disso ative o sininho de notificações para não perder nenhum video. vamos começar;'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'MIQUEINHA CORNO CONFORMADO! #memes #humor #amelhorredpillvemdelas #comedia seja bem vindo a mais um vídeo do canal Não se esqueça inscrever no CANAL e deixar o like  além disso ative o sininho de notificações para não perder nenhum vídeo.MIQUEINHA CORNO CONFORMADO!'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Mulher Rica casa com homem pobre? #homem #homens #homemdevalor #homemmoderno #motivacao Mulher Rica quer homem pobre? Nunca mais seja enganado, saiba todos os truques usados por elas neste link 👉 hotm.art/Antitrouxa 👈#homem #homens #motivacao #homemmoderno  #homemdevalor'"},
                {'role': 'assistant', 'content': "0"},
            ],
            'amostra_2': [
                {'role': 'user', 'content': "Text: 'HOMENS NÃO QUEREM CASAR #relacionamento #namoro #casamentonapraia #casal #masculino #macho#seduçao #machoalfa #redpilled #masculinidade #homemdevalor#redpillbrasil #namorosafado #conquistarmulheres #reconquistar #casamento#conquistarmulher #conquistarumamor #fabricioacunha #fabricioacunhamentor #machomacho#amor #cortespodcast #mulher #homem #shorts'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Ser Engraçado Ajuda  a Conquistar M1N4S ? Novo Canal Alternativo - https://www.youtube.com/ -c7dQual Sua Atratividade Facial De 1-10? Obtenha Sua Analise 🔍😊👤💡📏https://abre.ai/iX8DAdiquira o Ebook Guia De Looksmaxxing De 10 Etapas  👔✨📚📝👩‍🎨👨‍🎨📖https://abre.ai/iX8EAdiquira o Ebook  Looksmaxxing De Maneira Errada GRATIS   👔✨📚📝👩‍🎨https://abre.ai/iX8HMe Siga No Instagram  👣👉📸😊https://urlgeni.us/instagram/N0KqWMe Siga No TikTok  👣👉🎵😊https://urlgeni.us/tiktok/xcIwWMe Siga No Face 👣👉📘😊https://urlgeni.us/facebook/HEnL'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'MIQUEINHA EMOCIONADO COMPRANDO PRESENTE DE NATAL PARA A RAINHA DO EGITO CONFIA MIQUEINHA! Olá! seja bem vindo a mais um video do canal, hoje vamos falar, sobre MIQUEINHA EMOCIONADO COMPRANDO PRESENTE DE NATAL PARA A RAINHA DO EGITO CONFIA MIQUEINHA!não se esqueça de se inscrever no canal e deixar o like  além disso ative o sininho de notificações para não perder nenhum video. vamos começar;'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Bíceps no crossover. Forcei as últimas reps para cruzar do limite. Neste caso, a técnica n é a mesma '"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'Homem fuja de problema s #homem #homens #motivacao #homemmoderno #homemdevalor Quando ela tem isso cuidado homem #homem #homens #motivacao #homemmoderno #homemdevalor'"},
                {'role': 'assistant', 'content': "1"},
            ],
            'amostra_3': [
                {'role': 'user', 'content': "Text: 'Esposa me traia com o chefe. Vídeo completo no canal.'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Mulher Que Espera Príncipe Encantado🤦🏻'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'ESSA É A DIFERENÇA ENTRE AMOR E PAIXÃO Relacionamentos vs Redes Sociais: O Que Está Dando Errado? | Mari Vabo no Aqui com AmauryEpisódio completo: https://www.youtube.com/watch?v=q0nfvDSui3MCanal: Amaury Nuneshttps://www.youtube.com/ Convidados: Mariana Vabohttps://www.youtube.com/ - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 👑 Deixe o seu like e comentário, não se esqueça de inscrever-se no canal!📕 Aprenda como conquistar qualquer mulher. Acesse o link abaixo: https://bit.ly/47tJ0bQ - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - Instagram:   Tik Tok:   - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #desenvolvimentomasculino #seducao #homemdealtovalor #relacionamento #psicologiacomportamental #psicologaderelacionamento #inteligenciaemocional #linguagemcorporal'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: '4 Hábitos que vão TE TRANSFORMAR😃💪❤️'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'MSOL QUER UM MIQUEINHA PARA BANCAR ELA E OS FILHOS! CONFIA MIQUEINHA! #memes #amelhorredpillvemdelas Olá! seja bem vindo a mais um video do canal, hoje vamos falar, sobre o MSOL QUER UM MIQUEINHA PARA BANCAR ELA E OS FILHOS! CONFIA MIQUEINHA!. não se esqueça de se inscrever no canal e deixar o like  além disso ative o sininho de notificações para não perder nenhum video. vamos começar;'"},
                {'role': 'assistant', 'content': "1"},
            ],
            'amostra_4': [
                {'role': 'user', 'content': "Text: 'Isso é o que dá mais t3s  no homem | Thiago Schutz #shorts #reflexão #lar #amor #casamento #mulher'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'Peguei minha noiva mandando foto íntima para o meu pai, cabe dano moral? '"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Alguém me explica, tô meio perdido. Ela é femista e por isso é superior a nós? 🍷🗿 1- Adquira o AntiOtário clicando no site abaixo.👉🏻 https://hotm.art/ManualAnti 📕2- Domine as lições do nosso livro. 🍷3- Faça parte do nosso grupo privado. 🗿Te espero lá. 🫡AntiOtário, livro de desenvolvimento pessoal mais vendido do país.'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'CUIDADO COM ISSO QUE VAI FUNCIONAR #relacionamento #namoro #casamentonapraia #casal #masculino #relacionamento #namoro #casamentonapraia #casal #masculino #macho#seduçao #machoalfa #redpilled #masculinidade #homemdevalor#redpillbrasil #namorosafado #conquistarmulheres #reconquistar #casamento#conquistarmulher #conquistarumamor #fabricioacunha #fabricioacunhamentor #machomacho#amor #cortespodcast #mulher #homem'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'MULHERES ACIMA DE 30 ANOS É SÓ AMIGA! #humor #memes #amelhorredpillvemdelas #comedia seja bem vindo a mais um vídeo do canal Não se esqueça inscrever no CANAL e deixar o like  além disso ative o sininho de notificações para não perder nenhum vídeo.MULHERES ACIMA DE 30 ANOS É SÓ AMIGA!'"},
                {'role': 'assistant', 'content': "1"},
            ],
            'amostra_5': [
                {'role': 'user', 'content': "Text: 'Todo homem gostaria de dizer kkk #homem #homens #motivacao #homemmoderno #homemdevalor'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'Como vencer uma discussão | Qual a sua opinião? #rafaelaires  #antiotario  #redpillbrasil'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'Mulher e💲ao mesmo tempo💰😍'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'SE O HOMEM DE 30 ANOS NÃO CASOU, NÃO CASA MAIS #relacionamento #namoro #casamentonapraia #casal #relacionamento #namoro #casamentonapraia #casal #masculino #macho#seduçao #machoalfa #redpilled #masculinidade #homemdevalor#redpillbrasil #namorosafado #conquistarmulheres #reconquistar #casamento#conquistarmulher #conquistarumamor #fabricioacunha #fabricioacunhamentor #machomacho#amor #cortespodcast #mulher #homem'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'MULHER QUE SE MOSTRA DEMAIS QUER DIZER QUE É CARENTE #relacionamento #namoro #casamentonapraia #relacionamento #namoro #casamentonapraia #casal #masculino #macho#seduçao #machoalfa #redpilled #masculinidade #homemdevalor#redpillbrasil #namorosafado #conquistarmulheres #reconquistar #casamento#conquistarmulher #conquistarumamor #fabricioacunha #fabricioacunhamentor #machomacho#amor #cortespodcast #mulher #homem'"},
                {'role': 'assistant', 'content': "1"},
            ],
        },
        'misoginia': {
            'amostra_1': [
                {'role': 'user', 'content': "Text: 'AS LEIS DEFENDEM AS MULHERES #relacionamento #namoro #casamentonapraia #casal #masculino #macho #relacionamento #namoro #casamentonapraia #casal #masculino #macho#seduçao #machoalfa #redpilled #masculinidade #homemdevalor#redpillbrasil #namorosafado #conquistarmulheres #reconquistar #casamento#conquistarmulher #conquistarumamor #fabricioacunha #fabricioacunhamentor #machomacho#amor #cortespodcast #mulher #homem'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Você casou com um anjo ou com um demônio? | Thiago Schutz #shorts #relacionamento #casamento #amor'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'NÃO ACEITE Mulher que Faz Isso, Você Vai Ficar Chocado! '"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Homem fuja de problema s #homem #homens #motivacao #homemmoderno #homemdevalor Quando ela tem isso cuidado homem #homem #homens #motivacao #homemmoderno #homemdevalor'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: '5 coisas que os homens secretamente gostariam que as mulheres soubessem 🔴 **5 COISAS QUE OS HOMENS SECRETAMENTE GOSTARIAM QUE AS MULHERES SOUBESSEM!**  Você já se perguntou o que os homens realmente pensam? Será que eles são tão simples assim ou escondem segredos que nunca revelam? No vídeo de hoje, vou te contar **5 verdades ocultas** que todo homem gostaria que você soubesse – e a última é a mais importante e poderosa de todas!  🔥 **Assista até o final e descubra:**  ✔️ Por que os homens não entendem indiretas  ✔️ O medo secreto que eles nunca confessam  ✔️ O que realmente faz um homem se apaixonar  ✔️ Como evitar os erros que afastam os homens  ✔️ O que os homens mais desejam em um relacionamento  💡 **Se você quer entender melhor os homens e melhorar seus relacionamentos, não perca esse vídeo!**  📌 **Inscreva-se no canal Red Pill Feminina** e ative as notificações para mais conteúdos como este!  📩 **Compartilhe com suas amigas**, porque elas também precisam saber dessas verdades!  #Relacionamentos #RedPillFeminina #EntendaOsHomens #Namoro #PsicologiaMasculina #ComoConquistar #RelacionamentoSaudável  ---### **PARTE 2 - Referências, obras e autores citados**  1. **John Gray** – *Homens São de Marte, Mulheres São de Vênus*  2. **Jordan Peterson** – *12 Regras para a Vida*  3. **James Bauer** – *His Secret Obsession*  4. **Mark Manson** – *A Sutil Arte de Ligar o F*da-se*  5. **David Deida** – *O Caminho do Homem Superior*  6. **Napoleon Hill** – *Mais Esperto Que o Diabo*  7. **Robert Glover** – *No More Mr. Nice Guy*  8. **Rollo Tomassi** – *O Manual do Homem Racional* e *The Rational Male*  9. **Gary Chapman** – *As 5 Linguagens do Amor*  ---### **PARTE 3 - Palavras-chave (500 caracteres)**  relacionamento, como entender os homens, red pill feminina, segredos dos homens, o que os homens gostam, como conquistar um homem, psicologia masculina, dicas de relacionamento, o que afasta os homens, como ser uma mulher desejada, como fazer um homem se apaixonar, relacionamento saudável, homens e mulheres, erros femininos, namoro, casamento, sedução feminina, por que os homens se afastam, como atrair um homem, como ser admirada, conquistar um homem, dicas amorosas, comportamento masculinoGRUPO TELEGRAM👇https://bit.ly/CanalRedpillfemininahttps://go.hotmart.com/Y97146592T?ap=c8bcCreditos Da Criação Do VideoRoteiro; Feito com Chat gpthttps://chatgpt.com/imagens Geradas com piclumenhttps://piclumen.com Capa do video feito com o canvahttps://www.canva.com/imagens da capa, fonte seaarthttps://www.seaart.ai/ptEdição completa do video, efeitos e animaçõeshttps://www.capcut.com/Voz do Video Marcushttps://www.capcut.com/'"},
                {'role': 'assistant', 'content': "0"},
            ],
            'amostra_2': [
                {'role': 'user', 'content': "Text: 'O que fazer com namorada dramática #VSM #relacionamentosEntre para o Super Aulão de VSM: http://bit.ly/3RUC29n Neste vídeo, Gabriel Amorim dá as melhores dicas para você esquecer sua ex-namorada e virar a página de uma vez por todas.'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'ELE PERDEU TUDO 📕- ANTIOTÁRIO: https://antiotarioprooficial.com/oferta/Instagram:   https://www.instagram.com/antiotariopro/Telegram: https://t.me/antiotarioproFacebook:  https://www.facebook.com/antiotariopro/'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'RELATO DE HIPERGAMIA FEMININA! HOMEM ENCONTRA MULHER E DEPOIS ELA FAZ O QUE SE TORNOU PADRÃO Mulheres fugindo de segundos encontros sem explicação acontem há décadas. Só que o fato do feminismo ter liberado o profano femnino em sua totalidade, esses casos estão se tornando padrão. Trago um relato de um homem bem sucedido que marcou um encontro com uma mulher e deixou agendado um segundo encontro. Quando ele chega em casa, percebe que ela o bloqueou de todas as redes sociais. Mais um caso comprovando que as mulheres não devem ser mais motivo de queima de neurônios. Elas se transformaram nisso. Nós homens não temos culpa. Vamos focar nossas mentes em outras coisas, porque as mulheres atuais já estão fora de qualquer entendimento.#relacionamento #encontro'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Você conhece esse tipo de pessoa? | Thiago Schutz #shorts #podcast #relacionamento #thiagoschutz'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'Raiam Santos Levou Fora De Uma TATUADA #hackeandoamatrix Hoje no moreno tatuado tem Raiam Santos tendo exposto por Xheli G, ela vazou prints de Raiam Santos xavecando ela, mas o curioso é que Xheli G tem muitas tatuagens, e Raiam Santos sempre declarou que não apreceia pessoas que usam tatuagens.         #hackeandoamatrix  #hackeandoamatrix #morenotatuado #homemdealtovalor⭐Seja membro para ter acesso a vídeos exclusívos por apenas $2,99 🤏     https://www.youtube.com/channel/UCDrGZXd8k06ifz0-gccIn0w/join-----------------------------------------------------------------------------------------------------------Esse é um canal de opinião, onde analiso dinâmicas sociais entre homens e mulheres do ponto de vista de um homem.Meu compromisso é dar minha contribuição sincera e respeitosa sobre assuntos da atualidade que interessam ao homem de alto valor.Quando o homem começa a seguir o próprio caminho aprende que precisa se manter bem informado.💊DEIXE O LIKE!                       💊SE INSCREVA!             👉Compartilhe esse vídeo com alguém que vc gosta:Se Isso For Verdade Raiam Santos é o MAIOR GADO de Todos os Temposhttps://youtu.be/f47FcnC0cqM'"},
                {'role': 'assistant', 'content': "1"},
            ],
            'amostra_3': [
                {'role': 'user', 'content': "Text: 'O homem espera ISSO da mulher...'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Posso casar com GP? | Thiago Schutz #shorts #reflexão #casamento #amor #sonho #relacionamento #gp'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Sou o primeiro da minha namorada. #rafaelaires #antiotario #redpill'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'Homem FOFO 🤦🏻 Homem de PEGADA 🔥👊🏽'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'MÃE REDPILL SALVA FILHO MIQUEINHA DE MSOL! Olá! seja bem vindo a mais um video do canal, hoje vamos falar, SOBRE MÃE REDPILL SALVA FILHO MIQUEINHA DE MSOL! não se esqueça de se inscrever no canal e deixar o like  além disso ative o sininho de notificações para não perder nenhum video. vamos começar;'"},
                {'role': 'assistant', 'content': "1"},
            ],
            'amostra_4': [
                {'role': 'user', 'content': "Text: 'Estou sentindo um vazio muito grande na minha vida #VSM #relacionamentosEntre para o Super Aulão de VSM: http://bit.ly/3RUC29n Neste vídeo, Gabriel Amorim dá as melhores dicas para você esquecer sua ex-namorada e virar a página de uma vez por todas.'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'Por Que Mulheres Só Valorizarão o Homem Quando Ele Estiver Com Outra? Algumas mulheres só percebem o valor de um homem quando ele está em outro relacionamento, talvez devido ao desejo de conquistar o que não têm ou por ciúmes. Isso pode ser um reflexo de insegurança ou da percepção de que o homem tem mais valor quando não está disponível. #Relacionamento #Masculinidade #FabrícioAcunha #Valorização #Ciúmes'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'COMO DESTRAVAR A SUA VIDA AFETIVA #destravar #vidaafetiva #relacionamentoamoroso ✅Quer DESTRAVAR a sua vida afetiva definitivamente?👇LINK: https://pay.kiwify.com.br/hEskh2i✅GUIA DA MULHER AUTOCONFIANTEhttps://karinefloriana.com.br/✅E-BOOK: GUIA PRÁTICO PARA AUMENTAR O AMOR-PRÓPRIO E MELHORAR A AUTOESTIMAhttps://pay.kiwify.com.br/SIiljFS💎Comunidade Mulher Autoconfiante-Receba dicas no Telegramhttps://t.me/+gvhTCF_GHEA2Y2Qx🔻Me Encontre nas Redes Sociais🔻📸Instagram➤https://www.instagram.com/karinefloriana/📸Tiktok➤http://www.tiktok.com/ 🎬YouTube➤https://www.youtube.com/ 📸Linkedin➤www.linkedin.com/in/karinefloriana📸KWAI➤ https://kwai-video.com/u/ /n8XhvCfK✅ COMO DESTRAVAR A SUA VIDA AFETIVA Neste vídeo vamos falar sobre como destravar a sua vida afetiva utilizando a inteligência emocional para superar qualquer trava emocional que esteja te impedindo de viver um relacionamento saudável. Se você sente que tem algum bloqueio emocional amoroso, aqui você vai aprender a desbloquear sua mente e suas emoções para finalmente viver um relacionamento amoroso leve e verdadeiro.Descubra como identificar e eliminar os padrões que sabotam sua felicidade, conquistar um relacionamento equilibrado e entender como conquistar um homem sem medo e insegurança.📌 Descubra como destravar sua vida afetiva! Acesse o programa Destrava-se para o Amor e faça um teste exclusivo para identificar o que está bloqueando você. Clique no link no primeiro comentário fixado ou na descrição e comece hoje mesmo sua transformação.🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🔴Compartilhe esse vídeo : ➤ COMO DESTRAVAR A SUA VIDA AFETIVA https://youtube.com/shorts/qRdWZIq1PB8🚨Videos que também podem te ajudar🚨➤COMO DEIXAR UM HOMEM OBCECADO POR VOCÊ EM 6 PASSOS. AULA 1https://youtu.be/BZfEiq2sMpQ➤COMO CONQUISTAR UM HOMEM COM 4 ATITUDES? FAÇA ELE TE DESEJARhttps://youtu.be/dQBLREVfOmE➤COMO SE TORNAR EXTREMAMENTE ATRAENTE SEM ESFORÇOhttps://youtu.be/kzXwcEa2hOs➤TIPO DE MULHERES SOLTEIRAS QUE OS HOMENS NÃO GOSTAM -MULHERES DESINTERESSANTEShttps://youtu.be/NGwbYWizVjU➤COMO APRENDER A SER AMADA E VALORIZADA POR UM HOMEM RAPIDAMENTEhttps://youtu.be/DchFUqCx9TY➤COMO FAZER UM HOMEM FICAR INTERESSADO POR VOCÊ RAPIDAMENTE- DEIXE ELE APAIXONADOhttps://youtu.be/NsNIPOcIab0➤COMO CHAMAR A ATENÇÃO DE UM HOMEM?  FAÇA DIFERENTEhttps://youtu.be/6wvEfgkfwTI===========================================================🔴 #relacionamento #relacionamentoamoroso #comoconquistarumhomem #conquista #ComoDestravarSuaVidaAfetiva #DesbloquearVidaAfetiva #RelacionamentoAmoroso #DesbloqueioAfetivo #RelacionamentoSaudável #Autossabotagem #ComoConquistarUmHomem#ComoDestravarSuaVidaAfetiva #InteligenciaEmocional #RelacionamentoAmoroso #TravaEmocional #DesbloqueioEmocional #RelacionamentoSaudável'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'ELON MUSK: VILÃO OU HERÓI DA CIÊNCIA? Part. Felipe Hime ELON MUSK: VILÃO OU HERÓI DA CIÊNCIA? Part. Felipe Hime▶ LEPRECHAUN - Clique e acesse a plataformahttp://leprechaun.app/▶KOTAI PROJECThttps://www.kotaiproject.com/▶ DINOXXhttps://dinoxx.com.br/redcast/▶ BATISMO BITCOINhttps://payfast.greenn.com.br/redirect/113648/?a_id=913455.▶ LAC CLINICA https://lp1.lacclinica.com.br/▶ 👮🏻‍♂️ INSTITUTO ÓLIVER CARREIRAS POLICIAIS 👮🏻‍♂️👨🏻‍💻 Site Oficial 👉🏻 https://institutooliver.com.br/ 👩🏻‍🎓 Curso EJA em 3 meses 👉🏻 https://go.hotmart.com/S28462720M 👨🏻‍🏫 Curso Superior Sequencial em 3 meses 👉🏻 https://go.hotmart.com/Y11127534P 📞 34-993004408📶  ▶ Junior Masters veste Insider 👕 Cupom: REDCAST15Link: https://creators.insiderstore.com.br/Redcast▶ FTW SUPLEMENTOSUSE O CUMPOM REDFTW E GANHE 10% DE DESCONTOhttps://www.ftw.com.br/?parceiro=9755▶ LIVRO HACKEANDO O MERCADO SEXUALhttps://pay.kiwify.com.br/o0h7E4Y▶ INSTAGRAM DO HOST• Junior Masters:  ▶ INSTAGRAM DO REDCAST•   ▶ Junior Masters veste Insider 👕 Cupom: REDCAST15Link:https://www.insiderstore.com.br/?utm_medium=a64a8e2e&utm_campaign=FavoritosInsider&cupom=REDCASTBF&utm_source=influmkt-bf▶ LIVRO HACKEANDO O MERCADO SEXUALhttps://pay.kiwify.com.br/o0h7E4Y▶ INSTAGRAM DO HOST• Junior Masters:  ▶ INSTAGRAM DO REDCAST•   AS OPINIÕES, CONSIDERAÇÕES E COMENTÁRIOS EMITIDOS PELOS CONVIDADOS DO PROGRAMA, SERÃO ÚNICA E EXCLUSIVAMENTE RESPONSABILIDADE DE QUEM OS EMITIR.O REDCAST NÃO SE RESPONSABILIZA PELAS MESMAS.=====================================================Spotify: https://open.spotify.com/show/2qGNLUOtkA55qknCHicdoTYouTube Music: https://music.youtube.com/channel/UCeL1a4rpEA8UG9IQIewPccgAmazon Music: https://music.amazon.com.br/podcasts/5a492610-0c19-4087-9fde-a24f90421a10/redcastApple Podcasts: https://podcasts.apple.com/us/podcast/redcast/id1784860273'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'A mulher que tira a paz do Homem, ela perde esse Homem | Qual a sua opinião? #rafaelaires #antiotario #redpillbrasil🔥 Pare de ser feito de Otário! 🔥📕 Manual AntiOtário✅ Evitar manipulações e jogos emocionais✅ Se posicionar com inteligência nos relacionamentos✅ Torne-se a melhor versão de si mesmo📚 Adquira o seu agora! ⤵➡️| hotm.art/geracaoantiotario🚀 O livro que te ensina a se impor, evitar ciladas e nunca mais ser feito de trouxa.'"},
                {'role': 'assistant', 'content': "1"},
            ],
            'amostra_5': [
                {'role': 'user', 'content': "Text: 'Por Que Muitos Homens Continuam Buscando Outras Mulheres Mesmo Casados? Embora o casamento seja uma união de compromisso, muitos homens ainda podem se sentir atraídos por outras mulheres, seja pela busca por novas experiências, excitação ou até pela falta de satisfação emocional no relacionamento. No entanto, é importante que os relacionamentos sejam construídos com confiança, respeito e comunicação, para que ambos os parceiros possam expressar suas necessidades de maneira saudável. A chave para um casamento bem-sucedido é a transparência e o esforço mútuo. #Relacionamento #Masculinidade #FabrícioAcunha #Compromisso #Comunicação'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'O QUE VOCÊ NÃO DEVE FAZER NO PRIMEIRO ENCONTRO ✅O QUE VOCÊ NÃO DEVE FAZER NO PRIMEIRO ENCONTRONeste vídeo eu vou te explicar o que você não deve fazer no primeiro encontro.Vamos falar sobre o que fazer no primeiro encontro para conquistar um homem, ou melhor o que não fazer.Se você quer conquistar um homem , seduzir um homem, fazer um homem se apaixonar por você, então o primeiro encontro é fundamental para você conseguir conquistar esse homem. 🔴GUIA DA MULHER AUTOCONFIANTE🔴https://karinefloriana.com.br/🔴COMO RECONQUISTAR O EX🔴https://pay.kiwify.com.br/B5bvtUE🔴E-BOOK: GUIA PRÁTICO PARA AUMENTAR O AMOR-PRÓPRIO E MELHORAR A AUTOESTIMAhttps://pay.kiwify.com.br/SIiljFSINSCREVA-SE NO NOSSO CANAL PARA RECEBER MAIS CONSELHOS RELACIONADOS A RELACIONAMENTOS AMOROSO.🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🔴Compartilhe esse vídeo : ➤O QUE VOCÊ NÃO DEVE FAZER NO PRIMEIRO ENCONTROhttps://youtu.be/MDNLLoEDe0w🚨Videos que também podem te ajudar🚨➤COMO DESCOBRIR SE ELE TE QUER DE VERDADEhttps://youtu.be/KgiHeongySk➤COMO PARAR DE PENSAR NELE! 4 DICAS PARA MUDAR O PENSAMENTO E ESQUECER ELE.https://youtu.be/67d0yYyVIp8➤QUAIS SÃO AS CINCO PRINCIPAIS CARACTERÍSTICAS DE UM RELACIONAMENTO TÓXICOhttps://youtu.be/s4mve-Svf58➤3 ATITUDES QUE TODO HOMEM PROCURA EM UMA MULHER. SEJA DIFERENTE E IRRESISTÍVEL.https://youtu.be/4ldhbs-sq2E➤4 ATITUDES QUE AFASTAM OS HOMENShttps://youtu.be/sXr7rPLVb5Y➤4 COISAS QUE OS HOMENS ACHAM BONITO EM UMA MULHERhttps://youtu.be/2oARE1DH5Bw➤4 DICAS INFALÍVEIS DE COMO FAZER ELE SENTIR SUA FALTAhttps://youtu.be/p0Tp5mIxfsg➤SERÁ QUE ELE ME QUER? O HOMEM QUE TE QUER VAI FAZER ISTO.https://youtu.be/DzyypBZ0e6A➤3 COISAS QUE OS HOMENS AMAM NAS MULHERES. FAÇA ELE TE QUERERhttps://youtu.be/rMdqdIt8B6o➤GATILHOS VISUAIS QUE DEIXAM O HOMEM LOUCO, A ISCA PERFEITA.https://youtu.be/TUuBavtdsM0===========================================================🔴 #mulher #relacionamento #encontro #comoconquistarumhomem 🔻Me Encontre nas Redes Sociais🔻📸Instagram➤https://www.instagram.com/karinefloriana/📸Tiktok➤http://www.tiktok.com/ 🎬YouTube➤https://www.youtube.com/ 📸Linkedin➤www.linkedin.com/in/karinefloriana📸KWAI➤ https://kwai-video.com/u/ /n8XhvCfK'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'RELATO DE HIPERGAMIA FEMININA! HOMEM ENCONTRA MULHER E DEPOIS ELA FAZ O QUE SE TORNOU PADRÃO Mulheres fugindo de segundos encontros sem explicação acontem há décadas. Só que o fato do feminismo ter liberado o profano femnino em sua totalidade, esses casos estão se tornando padrão. Trago um relato de um homem bem sucedido que marcou um encontro com uma mulher e deixou agendado um segundo encontro. Quando ele chega em casa, percebe que ela o bloqueou de todas as redes sociais. Mais um caso comprovando que as mulheres não devem ser mais motivo de queima de neurônios. Elas se transformaram nisso. Nós homens não temos culpa. Vamos focar nossas mentes em outras coisas, porque as mulheres atuais já estão fora de qualquer entendimento.#relacionamento #encontro'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'NÃO NAMORE MULHER COM INSTAGRAM'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Como lidar com banirada de roupas curtas #VSM #relacionamentosEntre para o Super Aulão de VSM: http://bit.ly/3RUC29n Neste vídeo, Gabriel Amorim dá as melhores dicas para você esquecer sua ex-namorada e virar a página de uma vez por todas.'"},
                {'role': 'assistant', 'content': "1"},
            ],
        },
    },
    'fold_2': {
        'toxicidade': {
            'amostra_1': [
                {'role': 'user', 'content': "Text: 'MIQUEINHA CUIDADO COM A MULHER CAMALEÃO! #comedia #memes #amelhorredpillvemdelas #humor #redpill seja bem vindo a mais um vídeo do canal Não se esqueça inscrever no CANAL e deixar o like  além disso ative o sininho de notificações para não perder nenhum vídeo.MIQUEINHA CUIDADO COM A MULHER CAMALEÃO!'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Coitada da Luana Piovani #hackeandoamatrix Luana Piovani está comemorando 35 anos de carreira em portugal, cantando em restaurantes.⭐Seja membro para ter acesso a vídeos exclusívos por apenas $2,99 🤏     https://www.youtube.com/channel/UCDrGZXd8k06ifz0-gccIn0w/join     #hackeandoamatrix #morenotatuado #homemdealtovalor-----------------------------------------------------------------------------------------------------------Esse é um canal de opinião, onde analiso dinâmicas sociais entre homens e mulheres do ponto de vista de um homem.Meu compromisso é dar minha contribuição sincera e respeitosa sobre assuntos da atualidade que interessam ao homem de alto valor.Quando o homem começa a seguir o próprio caminho aprende que precisa se manter bem informado.💊DEIXE O LIKE!                       💊SE INSCREVA!             👉Compartilhe esse vídeo com alguém que vc gosta:O Que Tá Acontecendo Com a Luana Piovani ???https://youtu.be/srNomTLJovM'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'MULHER DIZ QUE ESTÁ NO PROCESSO DE CURA VEJA O RELATO! FIQUEI ATÉ EMOCIONADO! Olá! seja bem vindo a mais um video do canal, hoje vamos falar, sobre  MULHER DIZ QUE ESTÁ NO PROCESSO DE CURA VEJA O RELATO! FIQUEI ATÉ EMOCIONADO! não se esqueça de se inscrever no canal e deixar o like  além disso ative o sininho de notificações para não perder nenhum video. vamos começar;'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'NOVINHAS DO JOB! CONFIA MIQUEINHA! #relaciomento #humor #amelhorredpillvemdelas #memes Olá! seja bem vindo a mais um video do canal, hoje vamos falar, sobre o porquê que o homem deve malhar e treinar. não se esqueça de se inscrever no canal e deixar o like  além disso ative o sininho de notificações para não perder nenhum video. vamos começar;NOVINHAS DO JOB! CONFIA MIQUEINHA!'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'MUITA OSTENTAÇÃO, POUCO TRABALHO? JÁ SABE DE ONDE VEM O DINHEIRO! Ela vive postando viagens, bolsas caras, carros de luxo, restaurantes sofisticados… mas ninguém sabe de onde vem a renda. Neste vídeo, revelamos a verdade por trás de muitas dessas “vidas de princesa” na internet. Ostentação sem trabalho consistente geralmente esconde acordos por trás das câmeras. Se você é homem, fique atento. Nem tudo que brilha no feed é ouro — às vezes, é só “job” mesmo.'"},
                {'role': 'assistant', 'content': "1"},
            ],
            'amostra_2': [
                {'role': 'user', 'content': "Text: 'Mulheres modernas 🤦🏽'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'ELAS ESTÃO DESESPERADAS POIS NÃO TEM MAIS TEMPO A PERDER O INVERNO ESTÁ CHEGANDO PARA ELAS! seja bem vindo a mais um vídeo do canal,  ATENÇÃO hoje vamos ver!!ELAS ESTÃO DESESPERADAS POIS NÃO TEM MAIS TEMPO A PERDER O INVERNO ESTÁ CHEGANDO PARA ELAS! Não se esqueça inscrever no CANAL e deixar o like  além disso ative o sininho de notificações para não perder nenhum vídeo. vamos começar'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'A HONRADINHA ARRUINOU A MINHA VIDA - (Leonardo Torres  e Advogado Ciqueira) - Redcast #213 ▶ Junior Masters veste Insider 👕 Ganhe 12% OFF em todo o site usando o cupom REDCAST12! #insiderstoreAcesse: https://www.insiderstore.com.br/RedCast▶ TREINAMENTO BITCOIN BLACKPILLhttps://go.hotmart.com/E91436514A?dp=1▶ FTW SUPLEMENTOSUSE O CUMPOM RED E GANHE 10% DE DESCONTOhttps://www.ftw.com.br/?parceiro=9755▶ LIVRO HACKEANDO O MERCADO SEXUALhttps://pay.kiwify.com.br/o0h7E4Y▶ INSTAGRAM DO HOST• Junior Masters:  ▶ INSTAGRAM DO REDCAST•  ▶ PRODUÇÃO E GRAVAÇÃO ESTÚDIOS MCMinstagram:  www.estudiosmcm.com.br .AS OPINIÕES, CONSIDERAÇÕES E COMENTÁRIOS EMITIDOS PELOS CONVIDADOS DO PROGRAMA, SERÃO ÚNICA E EXCLUSIVAMENTE RESPONSABILIDADE DE QUEM OS EMITIR.O REDCAST NÃO SE RESPONSABILIZA PELAS MESMAS.'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Não pagou Lanche da Princesa! #homem #homens #motivacao #homemmoderno  #homemdevalor Namora 3 meses já quer Pensão #homem #homens #motivacao #homemmoderno  #homemdevalor'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'Minha esposa foi convidada p uma festa, onde mrdo N pode ir 📕 - AntiOtário 🔱 - Código Masculino🚩 - Red Flags🔞 - Antiporn*🧘 - Lições Perpétuas🔐 - Entrada no Grupo PrivadoEsse é apenas o começo! 📚🗿👇🔗 | https://hotm.art/ComboManual'"},
                {'role': 'assistant', 'content': "0"},
            ],
            'amostra_3': [
                {'role': 'user', 'content': "Text: '“Quem deve ficar junto são principalmente os iguais” #rafaelaires #antiotario #redpill 🗿 - Antiotario🔱 - Código Masculino🚩 - Red Flags🔞 - Antiporn*🧘 - Lições Perpétuas🔐 - Entrada no Grupo PrivadoEsse é apenas o começo! 📚🗿👇🔗 | https://hotm.art/ImperioCombo'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'PROJETO QUER ACABAR COM A PENSÃO SOCIOAFETIVA! E POR QUE ESSE PROJETO NÃO PODE PASSAR? EXPLICO! Muito louvável que parlamentares comecem a ficar atentos ao sistema feminista e proponham projetos. Porém, eles são imediatistas e não pensam a longo prazo. Na verdade, eles não entendem nada das implicações que os homens podem entrar. O projeto que quer acabar com a pensão socioafetiva é louvável pela coragem do proponente, no entanto, ele não percebe as implicações que os homens podem entrar e não entende da ainda persistente teimosia dos homens em assumir relacionamentos sérios. Os próprios homens são desconhecedores das leis.Não acredito que esse projeto passe e torço para que não aprovem, pois vai ter muitos homens que vão pensar que a coisa se resume apenas à pensão socioafetiva. https://www.otempo.com.br/politica/2025/2/20/deputado-do-pl-quer-acabar-com-pensao-alimenticia-socioafetiva-entenda-os-casos-afetados#relacionamento #pensaoalimenticia'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'SE A MULHER GOSTA DISSO NEM CHEGUE PERTO | MARI VABO Corte do Episódio: MARI VABO - REDCAST 71!Canal: REDCAST OFICIAL👑 Deixe o seu like e se possível comente!- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 👑 Descubra o poder do Método Alpha: método definitivo para transformar sua vida amorosa.  ⚜️ Aprenda os segredos da conquista, desenvolva sua confiança e se torne o homem que as mulheres desejam. ✅️ Com dicas práticas e estratégias comprovadas, e um passo a passo para se tornar o verdadeiro protagonista de sua jornada amorosa. 🔥 Liberte o seu potencial e conquiste o coração das mulheres com o Método Alpha. Chegou a hora de se destacar! Instagram:   Tik Tok:  #desenvolvimentomasculino #seducao #homemdealtovalor #relacionamento'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'SOLTEIRA TENTANDO JUSTIFICAR SUA SOLIDÃO MAS SENHORES TODA MULHER SONHA EM SER MÃE E TER FAMILIA! seja bem vindo a mais um vídeo do canal, hoje vamos ver  SOLTEIRA TENTANDO JUSTIFICAR SUA SOLIDÃO MAS SENHORES TODA MULHER SONHA EM SER MÃE E TER FAMILIA!  Não se esqueça inscrever no CANAL e deixar o like  além disso ative o sininho de notificações para não perder nenhum vídeo. vamos começar'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Como lidar com banirada de roupas curtas #VSM #relacionamentosEntre para o Super Aulão de VSM: http://bit.ly/3RUC29n Neste vídeo, Gabriel Amorim dá as melhores dicas para você esquecer sua ex-namorada e virar a página de uma vez por todas.'"},
                {'role': 'assistant', 'content': "0"},
            ],
            'amostra_4': [
                {'role': 'user', 'content': "Text: 'NÃO COMPARE COM O ANTES  #homemdealtovalor #podcast #relaciomento #seducao 📕 Descubra o poder do Método Alpha: método definitivo para transformar sua vida amorosa. Acesse o link abaixo:https://bit.ly/47tJ0bQ- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - Instagram:   Tik Tok:  - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #desenvolvimentomasculino #seducao #homemdealtovalor #relacionamento'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'MULHER SÓ USAM  ROUPAS CURTAS #relacionamento #namoro #casamentonapraia #casal #masculino #macho#seduçao #machoalfa #redpilled #masculinidade #homemdevalor#redpillbrasil #namorosafado #conquistarmulheres #reconquistar #casamento#conquistarmulher #conquistarumamor #fabricioacunha #fabricioacunhamentor #machomacho#amor #cortespodcast #mulher #homem #shorts'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Isso é fruto do seu trabalho, do seu suor 🗿 - Antiotario🔱 - Masculino🚩 - Red Flags🔞 - Antiporn*🧘 - Lições Perpétuas🔐 - Entrada no Grupo PrivadoEsse é apenas o começo! 📚🗿👇🔗 | https://hotm.art/ImperioCombo'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: '“A nossa sociedade está se transformando em um ambiente inóspito para relacionamento” 🍷🗿 Senhores…1. Obtenha o Mega Combo 📕2. Solicite sua entrada no grupo privado 🔐3. Desfrute de toda a sabedoria 🍷🗿Te espero lá ⤵🔗 - https://hotm.art/ComboManual'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: '#homem #homens #motivação Homem elas queriam ter dado mais'"},
                {'role': 'assistant', 'content': "1"},
            ],
            'amostra_5': [
                {'role': 'user', 'content': "Text: 'CURTIU A VIDA ADOIDADO NO CARROSSEL AGORA QUER CHORAR NA INTERNET!  MIQUEINHA , VAI QUE VAI DÁ BOM! Olá! seja bem vindo a mais um vídeo do canal. não se esqueça de se inscrever no canal e deixar o like  além disso ative o sininho de notificações para não perder nenhum vídeo. vamos começar;'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'VINGADORES PODCAST: SACANI, KAIO D`ELAQUA, JUNIOR, PETRY ▶ DINOXXhttps://dinoxx.com.br/redcast/▶ BATISMO BITCOINhttps://payfast.greenn.com.br/redirect/113648/?a_id=913455.▶ LAC CLINICA https://lp1.lacclinica.com.br/▶ 👮🏻‍♂️ INSTITUTO ÓLIVER CARREIRAS POLICIAIS 👮🏻‍♂️👨🏻‍💻 Site Oficial 👉🏻 https://institutooliver.com.br/ 👩🏻‍🎓 Curso EJA em 3 meses 👉🏻 https://go.hotmart.com/S28462720M 👨🏻‍🏫 Curso Superior Sequencial em 3 meses 👉🏻 https://go.hotmart.com/Y11127534P 📞 34-993004408📶  ▶ Junior Masters veste Insider 👕 Cupom: REDCAST15Link: https://creators.insiderstore.com.br/Redcast▶ FTW SUPLEMENTOSUSE O CUMPOM REDFTW E GANHE 10% DE DESCONTOhttps://www.ftw.com.br/?parceiro=9755▶ LIVRO HACKEANDO O MERCADO SEXUALhttps://pay.kiwify.com.br/o0h7E4Y▶ INSTAGRAM DO HOST• Junior Masters:  ▶ INSTAGRAM DO REDCAST•   ▶ BATISMO BITCOINhttps://payfast.greenn.com.br/redirect/113648/?a_id=913455.▶ 👮🏻‍♂️ INSTITUTO ÓLIVER CARREIRAS POLICIAIS 👮🏻‍♂️👨🏻‍💻 Site Oficial 👉🏻 https://institutooliver.com.br/ 👩🏻‍🎓 Curso EJA em 3 meses 👉🏻 https://go.hotmart.com/S28462720M 👨🏻‍🏫 Curso Superior Sequencial em 3 meses 👉🏻 https://go.hotmart.com/Y11127534P 📞 34-993004408📶  ▶ Junior Masters veste Insider 👕 Cupom: REDCAST15Link:https://www.insiderstore.com.br/?utm_medium=a64a8e2e&utm_campaign=FavoritosInsider&cupom=REDCASTBF&utm_source=influmkt-bf▶ FTW SUPLEMENTOSUSE O CUMPOM REDFTW E GANHE 10% DE DESCONTOhttps://www.ftw.com.br/?parceiro=9755▶ LIVRO HACKEANDO O MERCADO SEXUALhttps://pay.kiwify.com.br/o0h7E4Y▶ INSTAGRAM DO HOST• Junior Masters:  ▶ INSTAGRAM DO REDCAST•   AS OPINIÕES, CONSIDERAÇÕES E COMENTÁRIOS EMITIDOS PELOS CONVIDADOS DO PROGRAMA, SERÃO ÚNICA E EXCLUSIVAMENTE RESPONSABILIDADE DE QUEM OS EMITIR.O REDCAST NÃO SE RESPONSABILIZA PELAS MESMAS.=====================================================Spotify: https://open.spotify.com/show/2qGNLUOtkA55qknCHicdoTYouTube Music: https://music.youtube.com/channel/UCeL1a4rpEA8UG9IQIewPccgAmazon Music: https://music.amazon.com.br/podcasts/5a492610-0c19-4087-9fde-a24f90421a10/redcastApple Podcasts: https://podcasts.apple.com/us/podcast/redcast/id1784860273'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'O Simples que FUNCIONA...'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'Feministo detonando a REDPILL #mentalidadeblack #mestredablack #redpill #desenvolvimentopessoal #masculinidade ESSE CANAL trata da perspectiva MASCULINA do mundo moderno e DESENVOLVIMENTO PESSOAL opiniões sarcásticas e humoradaspeço respeito nos comentários'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Mulher Não Quer Homem Liso: Antes de Namorar, Seja Um Homem Bem-Sucedido Financeiramente! ✅👉  Se você gostou do meu conteúdo e quer ajudar o Canal, faça a sua doação aqui: 👇👇👇 (Vai me ajudar muito 🙏)https://livepix.gg/fabricioacunha✅👉 SERVIÇOS E CURSOS: Acesse o link abaixo 👇👇👇https://papodehomem.my.canva.site/fabricio✅👉 CURSO DE SEDUÇÃO [APRENDA A ARTE DA SEDUÇÃO] Acesse o link abaixo 👇👇👇https://payment.ticto.app/OADA42DF7✅👉RECONQUISTE A SUA EX DE VOLTAAcesse o link abaixo 👇👇👇https://payment.ticto.app/O42378241No vídeo de hoje, Fabrício Acunha fala sobre um tema essencial para qualquer homem que quer ter sucesso nos relacionamentos: a importância de ser financeiramente estável antes de se envolver com mulheres. Fabrício explica que mulher não quer homem liso, ou seja, homens que não têm a sua vida financeira bem encaminhada. Para que um relacionamento seja saudável e equilibrado, o homem precisa focar em conquistar sua liberdade financeira antes de pensar em conquistar mulheres.O vídeo mostra como um homem deve priorizar sua prosperidade e crescimento financeiro para garantir que está pronto para assumir um relacionamento de maneira madura. Fabrício alerta que o homem que está estabilizado financeiramente pode escolher a mulher que quiser, mas nunca gastar seu dinheiro com mulheres interesseiras. A verdadeira atração entre homem e mulher deve ser construída com base em respeito mútuo, valores e conexões emocionais, e não em vantagens financeiras.Fabrício também compartilha dicas sobre como encarar um relacionamento saudável. Ele ensina que antes de começar a se envolver com alguém, o homem precisa estar pronto para trazer estabilidade para o relacionamento, e isso começa com a sua própria estabilidade financeira. Ser próspero não é só sobre ganhar dinheiro, mas também sobre ter a confiança e o controle de sua vida, algo que atrai as mulheres de valor e permite que o homem tenha relações duradouras e sem manipulação.Assista ao vídeo completo para entender como ser um homem próspero e estabilizado financeiramente pode transformar a sua vida amorosa e garantir que você faça as escolhas certas para o seu futuro. Não se esqueça de deixar seu like, compartilhar com os amigos e se inscrever no canal para mais dicas sobre como ser um homem de alto valor.'"},
                {'role': 'assistant', 'content': "1"},
            ],
        },
        'misoginia': {
            'amostra_1': [
                {'role': 'user', 'content': "Text: '“O que mais impede de vocês evoluírem é vocês ficarem com qualquer toscanilda” 🍷🗿 Senhores…1. Obtenha o Mega Combo 📕2. Solicite sua entrada no grupo privado 🔐3. Desfrute de toda a sabedoria 🍷🗿Te espero lá ⤵🔗 - https://hotm.art/ComboManual'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'A menina de 11 e o namorado de 14 anos premeditaram para dar fim a mãe dela'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'ROBERTO JUSTOS PRECISA CONHECER A RED! Olá! seja bem vindo a mais um video do canal. não se esqueça de se inscrever no canal e deixar o like  além disso ative o sininho de notificações para não perder nenhum video. vamos começar;'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'Homem que assume MS0LL cuidado homem #homem #homens #motivacao #homemmoderno #homemdevalor Quando ela tem isso cuidado homem #homem #homens #motivacao #homemmoderno #homemdevalor'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'AS MULHERES PERSEGUEM TODO HOMEM QUE FAZ ISSO AUMENTE SEU VSM E TENHA A MULHER QUE QUISER:https://sedutorafro.com.br/aulaodevsm/'"},
                {'role': 'assistant', 'content': "1"},
            ],
            'amostra_2': [
                {'role': 'user', 'content': "Text: 'COMO SABER SE ELA É UMA MULHER DE BAIXO VALOR (LISTA altamente CANCELÁVEL) ⚜️ Você sabia que temos uma clube de assinatura reunindo o melhor do desenvolvimento pessoal masculino? Entre agora e faça parte do Conselho: https://bit.ly/49X2GXo⚜️ Deseja encontrar o seu propósito e encontrar sentido em tudo o que faz? Conheça as aulas de nosso curso Vocação e através deste conhecimento dê significado a sua vida. → https://bit.ly/3qw7PUk📚 Livros que indico• A História da Virilidade ➽ https://amzn.to/3QJBEs0• O Código dos Homens ➽ https://amzn.to/3JVDGD6• Disciplina é Liberdade ➽ https://amzn.to/3QmTXDz• Responsabilidade Extrema ➽ https://amzn.to/3pgHxRI• Hábitos Atômicos ➽ https://amzn.to/3SMBlyt• A Coragem de Não Agradar ➽ https://amzn.to/3B6IPnY• A Arte de Fazer Acontencer➽ https://amzn.to/3B7sL5D• Como Fazer Amigos E Influenciar Pessoas ➽ https://amzn.to/3qJmpVj• A Educação Da Vontade ➽ https://amzn.to/3DhWQSJ• A Vida Intelectual ➽ https://amzn.to/3QwDrQA• Como Ler Livros ➽ https://amzn.to/3Qpumt2• Como Estudar E Como Aprender ➽ https://amzn.to/3RJmpQE• O Ego É o Seu Inimigo ➽ https://amzn.to/3eC5eCe• A Arte De Viver Melhor ➽ https://amzn.to/3U8ua4y🎥 Ferramentas De Estudo:• Kindle 10a. geração ➽ https://amzn.to/3AknDvB• Caderno Preto Minimalista ➽ https://amzn.to/3xaI643• Bloco Notas Rápidas ➽ https://amzn.to/3AVkbXq🎥 Equipamentos que uso:• Webcam Logitech C920: https://amzn.to/3QGFP7X• Microfone Fifine K669B: https://amzn.to/3QlZHxj• Microfone lapela ECMLV1: https://amzn.to/3SMHhro• Monitor LG 24ML600M: https://amzn.to/3w3omyD• Ring Light 6 Polegadas 36: https://amzn.to/3PrLjm2• HD Externo 1TB USB 3.0 Seagate: https://amzn.to/3A1oRe2• Braço Articulado: https://amzn.to/3bQalxq• Bastão De Luz Brastoy: https://amzn.to/3QIWrfu★ Vitrine Completa de indicações: https://amzn.to/45ziGLW📲 Redes sociais:◼️ Telegram: https://t.me/projetoconselho◼️ Instagram: https://www.instagram.com/projetoconselho◼️ Twitter: https://twitter.com/projetoconselho✂️ Canal secundário: https://www.youtube.com/ ⭐ Quer conversar comigo?Traga suas questões, agende uma consultoria online: ➤ https://bit.ly/3QZ6Cjp◼️ Contato comercial E PIX ◼️➤ contatoprojetoconselho .com--  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --Sobre o canal:Bem-vindo ao Projeto Conselho, espaço de desenvolvimento pessoal masculino e inteligência emocional com o intuito de te ajudar a se tornar um homem melhor. Esse é o meu maior objetivo.'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'FALE UM MENTIRA QUE TODA MSOL CONTA! #memes #humor #comedia #redpil #redpillbrasil #redpill Olá! seja bem vindo a mais um video do canal, hoje vamos falar, sobre o porquê que o homem deve malhar e treinar. não se esqueça de se inscrever no canal e deixar o like  além disso ative o sininho de notificações para não perder nenhum video. vamos começar;FALE UM MENTIRA QUE TODA MSOL CONTA!'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Mulher Sem Valor (PARTE 2)🙅‍♂️'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Chegar em casa e estar tudo no lugar é sinônimo de felicidade, paz? 1- Adquira o Antiotário 📕2- Domine as lições do nosso livro 3- Faça parte do nosso grupo privado.➡️ https://manualantiotario.com/AntiOtário, livro de desenvolvimento pessoal mais vendido do país. 🫡'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'QUEM VAI GANHAR A ELEIÇÃO DE SP 2024? Ep. RED NO MBL! ▶ INSTITUTO OLIVERhttp://www.institutooliver.com/▶ Junior Masters veste Insider 👕 Ganhe 12% OFF em todo o site usando o cupom REDCAST12! #insiderstoreAcesse: https://www.insiderstore.com.br/RedCast▶ TREINAMENTO BITCOIN BLACKPILLhttps://go.hotmart.com/E91436514A?dp=1▶ FTW SUPLEMENTOSUSE O CUMPOM RED E GANHE 10% DE DESCONTOhttps://www.ftw.com.br/?parceiro=9755▶ LIVRO HACKEANDO O MERCADO SEXUALhttps://pay.kiwify.com.br/o0h7E4Y▶ INSTAGRAM DO HOST• Junior Masters:  ▶ INSTAGRAM DO REDCAST•  '"},
                {'role': 'assistant', 'content': "0"},
            ],
            'amostra_3': [
                {'role': 'user', 'content': "Text: 'Calma, Mateus! É só um body aberto, não é o fim do mundo 😂🩲 📕- ANTIOTÁRIO: https://antiotarioprooficial.com/oferta/Instagram:   https://www.instagram.com/antiotariopro/Telegram: https://t.me/antiotarioproFacebook:  https://www.facebook.com/antiotariopro/'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'Seu marido é assim? #homem #homens #homemdevalor #homemmoderno #motivacao Se ela falar isso abre o olho homem Nunca mais seja enganado, saiba todos os truques usados por elas neste link 👉 hotm.art/Antitrouxa 👈#homem #homens #motivacao #homemmoderno  #homemdevalor'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Mulher e💲ao mesmo tempo💰😍'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: '💥ATENÇÃO💥 PAZ 👉 SE NÃO TIVER ISSO, O CASAMENTO ACABA #PazMasculina #Relacionamento #Homem 💥 QUER VIRAR O JOGO? OS CURSOS ESTÃO AQUI ⬇Se você busca respeito, liberdade e autoridade.Homem que não aprende a prosperar… vai continuar sendo pisado, traído e ignorado.📚 CURSOS QUE VÃO ABRIR SUA MENTE ⬇✅ Curso 1 – A Regra do Jogo✅ Curso 2 – PROTOCOLO ANTIOTÁRIO - Rafael Aires✅ Curso 3 – Segredos dos Investimentos para Iniciantes✅ Curso 4 – Treinamento Bitcoin Black Pill (Renato 38itão)👉 Curso 1 – https://go.hotmart.com/D99197285G👉 Curso 2 – https://kiwify.app/0EvX8x9?afid=P0F8Xb4x👉 Curso 3 – https://go.hotmart.com/U99196870E👉 Curso 4 – https://go.hotmart.com/G93071926D ⚠️ Alerta para mulheres: NUNCA tire a paz de um homem! Entenda por que a paz é essencial para a felicidade e o bem-estar masculino e como a falta dela destrói relacionamentos.Este vídeo explora a necessidade profunda que os homens têm por paz em seus lares e relacionamentos. Descubra como conflitos constantes, cobranças e a falta de harmonia afetam a força, a motivação e até a saúde mental masculina. Aprenda a ser a parceira que oferece refúgio e apoio, e não mais um campo de batalha.Mulheres, reflitam sobre o poder que vocês têm! Compartilhem este vídeo e construam lares de paz!🔔 Se inscreve no canal, ativa o sininho e compartilha com um homem que precisa acordar.👊 Aqui é realidade, atitude e transformação.👑 Bem-vindo ao canal do Miquéinha.#PazParaHomens #HomemPrecisaDePaz #RelacionamentoSaudável #RespeitoMasculino #LarEmPaz #MulherSábia #HomemForte #Miqueinha #EntendendoOsHomens #HarmoniaNoLar'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'É POR ISSO QUE A MULHER SEMPRE TROCA VOCÊ EPISÓDIO 06 - RELACIONAMENTOS AFETIVOS Mari Vabo - Especialista em Relacionamentos #06Episódio completo: https://www.youtube.com/watch?v=e155w7w755ECanal: CARLOS HERDYhttps://www.youtube.com/ /featuredConvidada: Mariana Vabohttps://www.youtube.com/ - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 👑 Deixe o seu like e comentário, não se esqueça de inscrever-se no canal!📕 Aprenda como conquistar qualquer mulher. Acesse o link abaixo: https://bit.ly/47tJ0bQ - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - Instagram:   Tik Tok:   - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #desenvolvimentomasculino #seducao #homemdealtovalor #relacionamento #psicologiacomportamental #psicologaderelacionamento #inteligenciaemocional #linguagemcorporal'"},
                {'role': 'assistant', 'content': "0"},
            ],
            'amostra_4': [
                {'role': 'user', 'content': "Text: 'Como o casamento pode dar certo? | Thiago Schutz #shorts #reflexão #casamento #amor #mulher #fyp'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'ESSES SÃO OS DESEJOS OCULTOS DAS MULHERES “ESSES SÃO OS DESEJOS OCULTOS DAS MULHERES.”Corte do Episódio: MARI VABO NO BORA PODCAST (223) | DESVENDANDO OS SEGREDOS DAS MULHERESCanal: BORA PODCAST- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 👑 Descubra o poder do Método Alpha: método definitivo para transformar sua vida amorosa.  ⚜️ Aprenda os segredos da conquista, desenvolva sua confiança e se torne o homem que as mulheres desejam. ✅️ Com dicas práticas e estratégias comprovadas, e um passo a passo para se tornar o verdadeiro protagonista de sua jornada amorosa. 🔥 Liberte o seu potencial e conquiste o coração das mulheres com o Método Alpha. Chegou a hora de se destacar! Instagram:   Tik Tok:  #desenvolvimentomasculino #seducao #homemdealtovalor'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Cuidado com a Cultura Woke | A Nova Ditadura Disfarçada de Inclusão Cuidado com a Cultura Woke | A Nova Ditadura Disfarçada de InclusãoEles dizem que é sobre empatia.Sobre igualdade.Sobre justiça social.Mas se você ousar discordar… você é “cancelado”.A cultura woke não é sobre liberdade.É sobre controle.É sobre medo.É sobre reprogramar você pra aceitar tudo — até o absurdo — em nome da aceitação.Neste vídeo você vai entender:• Como a cultura woke sequestrou o debate, a verdade e a masculinidade• Por que o medo de ofender virou uma arma de censura disfarçada• E como homens fortes e conscientes se tornam inimigos naturais desse novo sistema de pensamentoAssista até o fim.Porque hoje você se cala pra “evitar treta”…Amanhã, será silenciado por pensar.---HASHTAGS:#CulturaWoke #SaiaDaMatrix #ControleSocial #LiberdadeDeExpressao #HomemDesperto #VelhoSábio #Cancelamento #VerdadeProibida #NovaDitadura---'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'SE VOCÊ É BONZINHO, ELA VAI TE USAR E DESCARTAR! 😤 Você já reparou como muitos homens bons são deixados de lado enquanto os “bad boys” estão sempre com as mulheres mais desejadas? Neste vídeo, vamos falar sobre a dura realidade: ser bonzinho demais pode fazer você ser visto como fraco, submisso e sem postura. Descubra como ter atitude sem perder sua essência — e por que as mulheres não se sentem atraídas por quem sempre diz “sim”.'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Essa semana eu conheci o novo tipo de esposa, a esposa do job '"},
                {'role': 'assistant', 'content': "1"},
            ],
            'amostra_5': [
                {'role': 'user', 'content': "Text: 'A dor da traição é Terrível #homem #homens #homemdevalor #homemmoderno #motivação Depois de Pular no Carrossel… Ela Quer Aliança Urgente | A Hipocrisia da “Nova Mulher”Ela passou anos “vivendo intensamente”…Colecionando experiências, validando ego e rejeitando os “certinhos”.Mas agora que o tempo cobrou a conta…Ela quer estabilidade.Quer compromisso.Quer um “homem de verdade”.E adivinha quem ela escolhe?Você.O cara que ela ignorou no passado.Neste vídeo, você vai descobrir:• O que é o “carrossel” da validação e por que tantos homens são usados como porto seguro após ele• Como o tempo age diferente no valor de mercado feminino e masculino• E por que você precisa parar de premiar quem te descartaria em outros temposAssista até o fim.Porque se você não entender esse ciclo,vai acabar criando filho alheio e pagando o preço da “fase louca” de alguém.#CarrosselFeminino #RelacionamentoModerno #VelhoSábio #HomemDesperto #VocêNãoÉPlanoB #SaiaDaMatrix #VerdadeProibida #AutoValorizaçãoMasculina #RedFlags'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'FAÇA ISSO QUANO ELE SE AFASTAR #amor #relacionamentosfelizes #relacionamento  #conquistaramado FAÇA ISSO QUANO ELE SE AFASTARNeste vídeo eu vou te explicar o que fazer quando ele se afasta, faça duas coisas para ele voltar. Se ele se afastou e você está se perguntando “ele se afastou de mim do nada”, “ele sumiu o que fazer?”, ele se afastou de mim”, assiste esse vídeo todo.Se ele se afastou nao procure, eu vou te explicar o que você deve fazer para ele voltar.🔴GUIA DA MULHER AUTOCONFIANTE🔴https://karinefloriana.com.br/🔴COMO RECONQUISTAR O EX🔴https://pay.kiwify.com.br/B5bvtUE🔴E-BOOK: 10 SEGREDOS PODEROSOS PARA AUMENTAR O AMOR-PRÓPRIO E POTENCIALIZAR SUA ATRAÇÃOhttps://pay.hotmart.com/C83942882PINSCREVA-SE NO NOSSO CANAL PARA RECEBER MAIS CONSELHOS RELACIONADOS A RELACIONAMENTOS E TÉCNICAS DE SEDUÇÃO.🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🔴Compartilhe esse vídeo : ➤FAÇA ISSO QUANO ELE SE AFASTAR https://youtube.com/shorts/129o1m5HeUc🚨Videos que também podem te ajudar🚨➤COMO DESCOBRIR SE ELE TE QUER DE VERDADEhttps://youtu.be/KgiHeongySk➤COMO PARAR DE PENSAR NELE! 4 DICAS PARA MUDAR O PENSAMENTO E ESQUECER ELE.https://youtu.be/67d0yYyVIp8➤QUAIS SÃO AS CINCO PRINCIPAIS CARACTERÍSTICAS DE UM RELACIONAMENTO TÓXICOhttps://youtu.be/s4mve-Svf58➤3 ATITUDES QUE TODO HOMEM PROCURA EM UMA MULHER. SEJA DIFERENTE E IRRESISTÍVEL.https://youtu.be/4ldhbs-sq2E➤4 ATITUDES QUE AFASTAM OS HOMENShttps://youtu.be/sXr7rPLVb5Y➤4 COISAS QUE OS HOMENS ACHAM BONITO EM UMA MULHERhttps://youtu.be/2oARE1DH5Bw➤4 DICAS INFALÍVEIS DE COMO FAZER ELE SENTIR SUA FALTAhttps://youtu.be/p0Tp5mIxfsg➤SERÁ QUE ELE ME QUER? O HOMEM QUE TE QUER VAI FAZER ISTO.https://youtu.be/DzyypBZ0e6A➤3 COISAS QUE OS HOMENS AMAM NAS MULHERES. FAÇA ELE TE QUERERhttps://youtu.be/rMdqdIt8B6o➤GATILHOS VISUAIS QUE DEIXAM O HOMEM LOUCO, A ISCA PERFEITA.https://youtu.be/TUuBavtdsM0===========================================================🔴 #relacionamento  #amor  #homem   #comoconquistarumhomem 🔻Me Encontre nas Redes Sociais🔻📸Instagram➤https://www.instagram.com/karinefloriana/📸Tiktok➤http://www.tiktok.com/ 🎬YouTube➤https://www.youtube.com/ 📸Linkedin➤www.linkedin.com/in/karinefloriana📸KWAI➤ https://kwai-video.com/u/ /n8XhvCfK'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'WAGNER THOMAZONI (TRAGICÔMICO) - REDCAST! ▶ BATISMO BITCOINhttps://payfast.greenn.com.br/redirect/113648/?a_id=913455.A SpikeToSpike é um ecossistema P2P, feito para conectar pessoas interessadas em negociar Bitcoin de maneira segura e com respeito à privacidade dos usuários.Usando o referral do RedCast você tem 20% de desconto nas taxas. Link na descrição ou QR code na tela.https://spiketospike.com/?referral=REDCAST▶ 👮🏻‍♂️ INSTITUTO ÓLIVER CARREIRAS POLICIAIS 👮🏻‍♂️👨🏻‍💻 Site Oficial 👉🏻 https://institutooliver.com.br/ 👩🏻‍🎓 Curso EJA em 3 meses 👉🏻 https://go.hotmart.com/S28462720M 👨🏻‍🏫 Curso Superior Sequencial em 3 meses 👉🏻 https://go.hotmart.com/Y11127534P 📞 34-993004408📶  ▶ Junior Masters veste Insider 👕 Cupom: REDCAST15Link: https://creators.insiderstore.com.br/Redcast▶ FTW SUPLEMENTOSUSE O CUMPOM REDFTW E GANHE 10% DE DESCONTOhttps://www.ftw.com.br/?parceiro=9755▶ LIVRO HACKEANDO O MERCADO SEXUALhttps://pay.kiwify.com.br/o0h7E4Y▶ INSTAGRAM DO HOST• Junior Masters:  ▶ INSTAGRAM DO REDCAST•   ▶ INSTITUTO OLIVERhttp://www.institutooliver.com/▶ Junior Masters veste Insider 👕 Ganhe 12% OFF em todo o site usando o cupom REDCAST12! #insiderstoreAcesse: https://www.insiderstore.com.br/RedCast▶ TREINAMENTO BITCOIN BLACKPILLhttps://go.hotmart.com/E91436514A?dp=1▶ FTW SUPLEMENTOSUSE O CUMPOM RED E GANHE 10% DE DESCONTOhttps://www.ftw.com.br/?parceiro=9755▶ LIVRO HACKEANDO O MERCADO SEXUALhttps://pay.kiwify.com.br/o0h7E4Y▶ INSTAGRAM DO HOST• Junior Masters:  ▶ INSTAGRAM DO REDCAST•   .AS OPINIÕES, CONSIDERAÇÕES E COMENTÁRIOS EMITIDOS PELOS CONVIDADOS DO PROGRAMA, SERÃO ÚNICA E EXCLUSIVAMENTE RESPONSABILIDADE DE QUEM OS EMITIR.O REDCAST NÃO SE RESPONSABILIZA PELAS MESMAS.'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: '“O homem não pode ser dono de casa” 🗿 - Antiotario🔱 - Masculino🚩 - Red Flags🔞 - Antiporn*🧘 - Lições Perpétuas🔐 - Entrada no Grupo PrivadoEsse é apenas o começo! 📚🗿👇🔗 | https://hotm.art/ImperioCombo'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: '“Ele se envolveu com uma toscanilda que tem 2 filhas e após 9 anos ela quis tirar tudo dele” 🍷🗿 Senhores…1. Obtenha o Mega Combo 📕2. Solicite sua entrada no grupo privado 🔐3. Desfrute de toda a sabedoria 🍷🗿Te espero lá ⤵🔗 - https://hotm.art/ComboManual'"},
                {'role': 'assistant', 'content': "1"},
            ],
        },
    },
    'fold_3': {
        'toxicidade': {
            'amostra_1': [
                {'role': 'user', 'content': "Text: 'AS MULHERES QUANDO QUEREM O CHAD FAZ DE TUDO! #memes #humor seja bem vindo a mais um vídeo do canal Não se esqueça inscrever no CANAL e deixar o like  além disso ative o sininho de notificações para não perder nenhum vídeo.AS MULHERES QUANDO QUEREM O CHAD FAZ DE TUDO!'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'MIQUEINHA SONHANDO QUE VAI ENCONTRAR A NAMORADA PERFEITA! #memes #humor #comedia seja bem vindo a mais um vídeo do canal Não se esqueça inscrever no CANAL e deixar o like  além disso ative o sininho de notificações para não perder nenhum vídeo.MIQUEINHA SONHANDO QUE VAI ENCONTRAR A NAMORADA PERFEITA!'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Quem Se Importa mais com a Aparência? Homens ou Mulheres ? ... Clube De Membos Conteúdo Sem Censura Multiplataforma - https://abre.ai/maOVSua Atratividade Facial De 1-10? Obtenha Sua Analise 🔍😊👤💡📏https://abre.ai/iX8DAdiquira o Ebook Guia De Looksmaxxing De 10 Etapas  👔✨📚📝👩‍🎨👨‍🎨📖https://abre.ai/iX8EAdiquira o Ebook  Looksmaxxing De Maneira Errada GRATIS   👔✨📚📝👩‍🎨https://abre.ai/iX8HMe Siga No Instagram  👣👉📸😊https://urlgeni.us/instagram/N0KqWMe Siga No TikTok  👣👉🎵😊https://urlgeni.us/tiktok/xcIwWMe Siga No Face 👣👉📘😊https://urlgeni.us/facebook/HEnL'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'No futuro elas vão agradecer o pai e a mãe que tem 📕 - AntiOtário 🔱 - Código Masculino🚩 - Red Flags🔞 - Antiporn*🧘 - Lições Perpétuas🔐 - Entrada no Grupo PrivadoEsse é apenas o começo! 📚🗿👇🔗 | https://hotm.art/ComboManual'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'MULHER COMPROMETIDA NÃO FICA POSTANDO FOTO #relacionamento #namoro #casamentonapraia #casal #relacionamento #namoro #casamentonapraia #casal #masculino #macho#seduçao #machoalfa #redpilled #masculinidade #homemdevalor#redpillbrasil #namorosafado #conquistarmulheres #reconquistar #casamento#conquistarmulher #conquistarumamor #fabricioacunha #fabricioacunhamentor #machomacho#amor #cortespodcast #mulher #homem'"},
                {'role': 'assistant', 'content': "1"},
            ],
            'amostra_2': [
                {'role': 'user', 'content': "Text: 'Atenção com as exigências mulheres! | Qual a sua opinião? #rafaelaires  #antiotario  #redpillbrasil'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'SE VOCÊ NÃO TEM DINHEIRO, VOCÊ NÃO É NINGUÉM #relacionamento #sedução #namoro #conquistarmulheres '"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'TODA MULHER FICA ASSIM QUANDO TRAI O MARIDO! #memes #humor #amelhorredpillvemdelas #comedia seja bem vindo a mais um vídeo do canal Não se esqueça inscrever no CANAL e deixar o like  além disso ative o sininho de notificações para não perder nenhum vídeo.TODA MULHER FICA ASSIM QUANDO TRAI O MARIDO!'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Meu filho é grosseiro, mal educado e me trata muito mal, só tem 15 anos, o que devo fazer? '"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'COMO DESCOBRIR UM HOMEM DE VALOR? ✅COMO DESCOBRIR UM HOMEM DE VALOR?Neste vídeo eu vou te responder a questão, será que não existe homem de valor?, onde encontrar um homem de valor? E como descobrir um homem de valor?Se você busca um homem de valor para ter um relacionamento sério e  acha que não existe mais homemdevalor,  homem de alto valor  ou homem de verdade neste vídeo eu vou te dar as respostas que você precisa.Também vou falar sobre como encontrar um homem de valor  e onde encontrar homens de valor?🔴GUIA DA MULHER AUTOCONFIANTE🔴https://karinefloriana.com.br/🔴COMO RECONQUISTAR O EX🔴https://pay.kiwify.com.br/B5bvtUE🔴E-BOOK: 10 SEGREDOS PODEROSOS PARA AUMENTAR O AMOR-PRÓPRIO E POTENCIALIZAR SUA ATRAÇÃOhttps://pay.kiwify.com.br/SIiljFSINSCREVA-SE NO NOSSO CANAL PARA RECEBER MAIS CONSELHOS RELACIONADOS A RELACIONAMENTOS E TÉCNICAS DE SEDUÇÃO.🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🔴Compartilhe esse vídeo : ➤COMO DESCOBRIR UM HOMEM DE VALOR?https://youtu.be/kjdYs5or60k🚨Videos que também podem te ajudar🚨➤COMO DESCOBRIR SE ELE TE QUER DE VERDADEhttps://youtu.be/KgiHeongySk➤COMO PARAR DE PENSAR NELE! 4 DICAS PARA MUDAR O PENSAMENTO E ESQUECER ELE.https://youtu.be/67d0yYyVIp8➤QUAIS SÃO AS CINCO PRINCIPAIS CARACTERÍSTICAS DE UM RELACIONAMENTO TÓXICOhttps://youtu.be/s4mve-Svf58➤3 ATITUDES QUE TODO HOMEM PROCURA EM UMA MULHER. SEJA DIFERENTE E IRRESISTÍVEL.https://youtu.be/4ldhbs-sq2E➤4 ATITUDES QUE AFASTAM OS HOMENShttps://youtu.be/sXr7rPLVb5Y➤4 COISAS QUE OS HOMENS ACHAM BONITO EM UMA MULHERhttps://youtu.be/2oARE1DH5Bw➤4 DICAS INFALÍVEIS DE COMO FAZER ELE SENTIR SUA FALTAhttps://youtu.be/p0Tp5mIxfsg➤SERÁ QUE ELE ME QUER? O HOMEM QUE TE QUER VAI FAZER ISTO.https://youtu.be/DzyypBZ0e6A➤3 COISAS QUE OS HOMENS AMAM NAS MULHERES. FAÇA ELE TE QUERERhttps://youtu.be/rMdqdIt8B6o➤GATILHOS VISUAIS QUE DEIXAM O HOMEM LOUCO, A ISCA PERFEITA.https://youtu.be/TUuBavtdsM0===========================================================🔴 #homem #relacionamento #amor #homemdevalor #homemdeverdade 🔻Me Encontre nas Redes Sociais🔻📸Instagram➤https://www.instagram.com/karinefloriana/📸Tiktok➤http://www.tiktok.com/ 🎬YouTube➤https://www.youtube.com/ 📸Linkedin➤www.linkedin.com/in/karinefloriana📸KWAI➤ https://kwai-video.com/u/ /n8XhvCfK'"},
                {'role': 'assistant', 'content': "0"},
            ],
            'amostra_3': [
                {'role': 'user', 'content': "Text: 'Adoro #hackeandoamatrix Hoje no moreno tatuado tem Cinthia Chagas contando uma história inusitada de um de seus alunos que disse que quando via Cinthia usando um determinado tipo de bota, ele tinha pensamentos inadequados com ela.       #hackeandoamatrix #morenotatuado #homemdealtovalor⭐Seja membro para ter acesso a vídeos exclusívos por apenas $2,99 🤏     https://www.youtube.com/channel/UCDrGZXd8k06ifz0-gccIn0w/join-----------------------------------------------------------------------------------------------------------Esse é um canal de opinião, onde analiso dinâmicas sociais entre homens e mulheres do ponto de vista de um homem.Meu compromisso é dar minha contribuição sincera e respeitosa sobre assuntos da atualidade que interessam ao homem de alto valor.Quando o homem começa a seguir o próprio caminho aprende que precisa se manter bem informado.💊DEIXE O LIKE!                       💊SE INSCREVA!             👉Compartilhe esse vídeo com alguém que vc gosta:O Aluno Queria Me Ver P3LADA Só Usando Bota e CHICOTEhttps://youtube.com/shorts/juSnMKVzkmQhttps://youtu.be/_udK2Qz5rhg'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'O CHAD VADER DEIXA ELAS LOUCAS!'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'qUAL A VERDADEIRA RAZAO PELA QUAL OS HOMENS SUPERAM AS MULHERES NA SEDUÇÃO'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'MULHER ATACANDO A RED! CORRAM PARA AS COLINAS MEUS QUERIDO! VÃO SER MÃE DE PETS! seja bem vindo a mais um vídeo do canal Não se esqueça inscrever no CANAL e deixar o like  além disso ative o sininho de notificações para não perder nenhum vídeo.MULHER ATACANDO A RED! CORRAM PARA AS COLINAS MEUS QUERIDO! VÃO SER MÃE DE PETS!'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'A Importância de escolher bem seu/sua parceiro (a) | Qual a sua opinião? #rafaelaires  #antiotario  #redpillbrasil 🔥 Pare de ser feito de Otário! 🔥📕 Manual AntiOtário✅ Evitar manipulações e jogos emocionais✅ Se posicionar com inteligência nos relacionamentos✅ Torne-se a melhor versão de si mesmo📚 Adquira o seu agora! ⤵➡️| hotm.art/geracaoantiotario🚀 O livro que te ensina a se impor, evitar ciladas e nunca mais ser feito de trouxa. '"},
                {'role': 'assistant', 'content': "0"},
            ],
            'amostra_4': [
                {'role': 'user', 'content': "Text: 'Mulher que se exibe 🤦🏽'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'PRA TUDO TEM UM MAS...  #homemdealtovalor #podcast #seducao 👑 Descubra o poder do Método Alpha: método definitivo para transformar sua vida amorosa.  ⚜️ Aprenda os segredos da conquista, desenvolva sua confiança e se torne o homem que as mulheres desejam. ✅️ Com dicas práticas e estratégias comprovadas, e um passo a passo para se tornar o verdadeiro protagonista de sua jornada amorosa. 🔥 Liberte o seu potencial e conquiste o coração das mulheres com o Método Alpha. Chegou a hora de se destacar! Instagram:   Tik Tok:  #desenvolvimentomasculino #seducao #homemdealtovalor #relacionamento'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'Sinais que Deus me deu pra eu namorar meu namorado 📕 - AntiOtário 🔱 - Código Masculino🚩 - Red Flags🔞 - Antiporn*🧘 - Lições Perpétuas🔐 - Entrada no Grupo PrivadoEsse é apenas o começo! 📚🗿👇🔗 | https://hotm.art/ComboManual'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'MULHERES ACIMA DE 30 ANOS É SÓ AMIGA! #humor #memes #amelhorredpillvemdelas #comedia seja bem vindo a mais um vídeo do canal Não se esqueça inscrever no CANAL e deixar o like  além disso ative o sininho de notificações para não perder nenhum vídeo.MULHERES ACIMA DE 30 ANOS É SÓ AMIGA!'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'MIQUEINHA CORNO CONFORMADO! #memes #humor #amelhorredpillvemdelas #comedia seja bem vindo a mais um vídeo do canal Não se esqueça inscrever no CANAL e deixar o like  além disso ative o sininho de notificações para não perder nenhum vídeo.MIQUEINHA CORNO CONFORMADO!'"},
                {'role': 'assistant', 'content': "1"},
            ],
            'amostra_5': [
                {'role': 'user', 'content': "Text: 'Por que será homem? #homem #homens #homemdevalor #homemmoderno #motivação 🌟 **As 16 Leis do Sucesso de Napoleon Hill: Explicadas Detalhadamente** 🌟Neste episódio do nosso vídeo, vamos explorar as 16 Leis do Sucesso de Napoleon Hill, um dos maiores autores de desenvolvimento pessoal e motivação. Vamos analisar cada uma das leis e entender como elas podem ser aplicadas para alcançar seus objetivos e transformar sua vida.Desde a importância de um propósito definido até a prática da autoconfiança e a construção de uma mentalidade positiva, discutiremos como cada princípio contribui para o sucesso pessoal e profissional. Prepare-se para insights valiosos que podem impulsionar sua jornada em direção ao sucesso!✨ **Não se esqueça de curtir, compartilhar e se inscrever para mais conteúdos sobre desenvolvimento pessoal e motivação!** ✨### Hashtags:#NapoleonHill #LeisDoSucesso #DesenvolvimentoPessoal #Motivação #Sucesso #MentalidadePositiva #Podcast #CrescimentoPessoal Se precisar de mais alguma coisa, é só avisar!'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'Será Se Ainda 🤔Existe Mulher PRA CASAR? ➡️Meu Livro Atração Extraordinária - https://topseduzir.com/atracao-extraordinaria**********************************************❤️️❤️️Você Pode Gostar Desses Vídeos Também❤️️❤️️😜5 Coisas Que as Mulheres ADORAM e Poucos Homens Fazem:https://www.youtube.com/watch?v=kWoD8O89xsk😱10 Coisas Sobre as Mulheres Que Elas NÃO QUEREM Que Você Saiba:https://www.youtube.com/watch?v=KwOf8oKc5Dw&t=59s🌟Veja o Que ATRAI 99,9% das Mulheres e Você NEM IMAGINA:https://www.youtube.com/watch?v=J67gfW2Wrj8&t=31s💋A Coisa PRINCIPAL Que ATIVA o Instinto da Mulher de Querer Ficar Com Você:https://www.youtube.com/watch?v=vTZEwA2ajfc🧲️99% das Mulheres Irão Se Atrair Por Você Se Agir ASSIM:https://www.youtube.com/watch?v=oUDwKYqgsLY💖Playlist de Técnicas e Estratégias que CONQUISTAM:https://www.youtube.com/watch?v=JRMTFCJdN0U&list=PLmVxZj8cB-zyIrB_EZtJAzs5C9hyYa_q1&index=1&t=20s********************************************⏭SE CONECTE COMIGO⏮💻MEU SITE OFICIAL:https://topseduzir.com/✅MEU INSTAGRAM ********************************************📧CONTATO PARA DIVERSOS ASSUNTOS:ronaldogarcia .com*********************************************🎵CRÉDITO MUSICAL:Youtube'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'MIQUEINHA CORNO CONFORMADO! #memes #humor #amelhorredpillvemdelas #comedia seja bem vindo a mais um vídeo do canal Não se esqueça inscrever no CANAL e deixar o like  além disso ative o sininho de notificações para não perder nenhum vídeo.MIQUEINHA CORNO CONFORMADO!'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: '“A minha prima está querendo ter um filho meu e eu não vou dar” 🍷🗿 📕 - Antiotario🔱 - Código Masculino🚩 - Red Flags🔞 - Antiporn*🧘 - Lições Perpétuas🔐 - Entrada no Grupo PrivadoEsse é apenas o começo! 📚🗿👇🔗 | https://hotm.art/ComboManual'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'Se o homem não for líder, vai ter divórcio 🗿 - Antiotario🔱 - Masculino🚩 - Red Flags🔞 - Antiporn*🧘 - Lições Perpétuas🔐 - Entrada no Grupo PrivadoEsse é apenas o começo! 📚🗿👇🔗 | https://hotm.art/ImperioCombo'"},
                {'role': 'assistant', 'content': "1"},
            ],
        },
        'misoginia': {
            'amostra_1': [
                {'role': 'user', 'content': "Text: 'Marmita #hackeandoamatrix ⭐Seja membro para ter acesso a vídeos exclusívos por apenas $2,99 🤏     https://www.youtube.com/channel/UCDrGZXd8k06ifz0-gccIn0w/join    #hackeandoamatrix #morenotatuado #homemdealtovalor-----------------------------------------------------------------------------------------------------------Esse é um canal de opinião, onde analiso dinâmicas sociais entre homens e mulheres do ponto de vista de um homem.Meu compromisso é dar minha contribuição sincera e respeitosa sobre assuntos da atualidade que interessam ao homem de alto valor.Quando o homem começa a seguir o próprio caminho aprende que precisa se manter bem informado.💊DEIXE O LIKE!                       💊SE INSCREVA!             👉Compartilhe esse vídeo com alguém que vc gosta:Quem Não Se Emocionar Com a História Dessa Mãe Já Morreu Por Dentrohttps://youtu.be/2SPUlcLZvLg'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'JOB PRIMA RICA DANDO CONSELHOS FINANCEIROS! Olá! seja bem vindo a mais um video do canal, hoje vamos falar, sobre JOB PRIMA RICA DANDO CONSELHOS FINANCEIROS!. não se esqueça de se inscrever no canal e deixar o like  além disso ative o sininho de notificações para não perder nenhum video. vamos começar;'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'A FUTURA ESPOSA DO MIQUEINHA! E DOS SENHORES QUE NÃO SEGUIREM O CANAL! KKKKK Olá! seja bem vindo a mais um video do canal, hoje vamos falar, sobre A FUTURA ESPOSA DO MIQUEINHA! E DOS SENHORES QUE NÃO SEGUIREM O CANAL! KKKKKnão se esqueça de se inscrever no canal e deixar o like  além disso ative o sininho de notificações para não perder nenhum video. vamos começar;'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'Hulk casou com a sobrinha??? #hackeandoamatrix O casamento de Hulk, 38, e Camila Ângelo, 35, causou polêmica nas redes sociais no fim de semana. Eles celebraram a união em uma cerimônia religiosa na sexta-feira (3) e estão casados no civil desde 2020. O relacionamento começou em outubro de 2019, meses depois de o jogador de futebol terminar seu casamento com Iran Ângelo, que é tia de Camila. Hulk foi casado com Iran entre 2007 e 2019. Eles tiveram três filhos: Ian, de 14 anos; Tiago, de 12, e Alice, de 9. Hulk e Iran anunciaram o término em agosto.     Poucos meses depois, em outubro, o novo relacionamento de Hulk veio à tona. Ele passou a namorar Camila, que é sobrinha de Iran. Naquela época, a jornalista Fábia Oliveira divulgou uma carta que ela teria escrito para a tia, dizendo que eles tinham se permitido viver um sentimento que estava incubado… - Veja mais em https://www.uol.com.br/splash/noticias/2025/01/06/por-que-o-casamento-de-hulk-com-sobrinha-da-ex-causou-entenda-polemica.htm?cmpid=copiaecola⭐Seja membro para ter acesso a vídeos exclusívos por apenas $2,99 🤏     https://www.youtube.com/channel/UCDrGZXd8k06ifz0-gccIn0w/join#hackeandoamatrix #morenotatuado #homemdealtovalor-----------------------------------------------------------------------------------------------------------Esse é um canal de opinião, onde analiso dinâmicas sociais entre homens e mulheres do ponto de vista de um homem.Meu compromisso é dar minha contribuição sincera e respeitosa sobre assuntos da atualidade que interessam ao homem de alto valor.Quando o homem começa a seguir o próprio caminho aprende que precisa se manter bem informado.💊DEIXE O LIKE!                       💊SE INSCREVA!             👉Compartilhe esse vídeo com alguém que vc gosta:Hulk Se Casou Com a Própria Sobrinha e Insinuou Que a Ex Esposa Era do JOBhttps://youtu.be/OZhNsIwQVF4'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'MULHER EXIGENTE FICA SOZINHA #relacionamento #namoro #casamentonapraia #casal #masculino #macho#seduçao #machoalfa #redpilled #masculinidade #homemdevalor#redpillbrasil #namorosafado #conquistarmulheres #reconquistar #casamento#conquistarmulher #conquistarumamor #fabricioacunha #fabricioacunhamentor #machomacho#amor #cortespodcast #mulher #homem #shorts'"},
                {'role': 'assistant', 'content': "1"},
            ],
            'amostra_2': [
                {'role': 'user', 'content': "Text: 'Dá para namorar quem bebe? | Thiago Schutz #shorts #reflexão #relacionamento #mulher #fyp #wow'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: '3 obrigações das mulheres africanas'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'TODA MULHER FICA ASSIM QUANDO TRAI O MARIDO! #memes #humor #amelhorredpillvemdelas #comedia seja bem vindo a mais um vídeo do canal Não se esqueça inscrever no CANAL e deixar o like  além disso ative o sininho de notificações para não perder nenhum vídeo.TODA MULHER FICA ASSIM QUANDO TRAI O MARIDO!'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Receita pra afastar o homem🤦🏽'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'MULHER REFUTANDO O FEMINISMO COM ARGUMENTOS! seja bem vindo a mais um vídeo do canal, hoje vamos SOBRE MULHER REFUTANDO O FEMINISMO COM ARGUMENTOS! não se esqueça de se inscrever no canal e deixar o like  além disso ative o sininho de notificações para não perder nenhum vídeo. vamos começar'"},
                {'role': 'assistant', 'content': "0"},
            ],
            'amostra_3': [
                {'role': 'user', 'content': "Text: '“O que você acha do meio liberal?”'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'Filho você se casaria com uma mulher assim? 1- Adquira o Antiotário 📕2- Domine as lições do nosso livro 3- Faça parte do nosso grupo privado.➡️ https://manualantiotario.com/AntiOtário, livro de desenvolvimento pessoal mais vendido do país. 🫡'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Se não for pra ter uma mulher virtuosa e de valor, melhor ficar na sua. | Qual a sua opinião? #rafaelaires #antiotario #redpillbrasil🔥 Pare de ser feito de Otário! 🔥📕 Manual AntiOtário✅ Evitar manipulações e jogos emocionais✅ Se posicionar com inteligência nos relacionamentos✅ Torne-se a melhor versão de si mesmo📚 Adquira o seu agora! ⤵➡️| hotm.art/geracaoantiotario🚀 O livro que te ensina a se impor, evitar ciladas e nunca mais ser feito de trouxa.'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'MULHER FALA PARA ELAS SEREM TÓXICAS! seja bem vindo a mais um vídeo do canal, MULHER FALA PARA ELAS SEREM TÓXICAS! hoje vamos ver! Não se esqueça inscrever no CANAL e deixar o like  além disso ative o sininho de notificações para não perder nenhum vídeo. vamos começar'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'MULHER FALA QUE HOMEM NÃO PRESTA CONFIA MIQUEINHA! Olá! seja bem vindo a mais um video do canal, hoje vamos falar, sobre MULHER FALA QUE HOMEM NÃO PRESTA CONFIA MIQUEINHA! não se esqueça de se inscrever no canal e deixar o like  além disso ative o sininho de notificações para não perder nenhum video. vamos começar;'"},
                {'role': 'assistant', 'content': "0"},
            ],
            'amostra_4': [
                {'role': 'user', 'content': "Text: 'Por que se manter longe de mulher recém solteira #VSM #relacionamentosEntre para o Super Aulão de VSM: http://bit.ly/3RUC29n Neste vídeo, Gabriel Amorim dá as melhores dicas para você esquecer sua ex-namorada e virar a página de uma vez por todas.'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'MULHER FALANDO QUE ELAS TEM QUE APRENDER A VIVER SEM HOMEM! Olá! seja bem vindo a mais um video do canal, hoje vamos falar, sobre MULHER FALANDO QUE ELAS TEM QUE APRENDER A VIVER SEM HOMEM! não se esqueça de se inscrever no canal e deixar o like  além disso ative o sininho de notificações para não perder nenhum video. vamos começar;'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'Você não trabalha, só fica em casa 1- Adquira o Antiotário 📕2- Domine as lições do nosso livro 3- Faça parte do nosso grupo privado.➡️ https://manualantiotario.com/AntiOtário, livro de desenvolvimento pessoal mais vendido do país. 🫡'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'MULHER INCENTIVANDO A TIRAR DINHEIRO DOS HOMENS! CORRA PARA AS COLINAS! Olá! seja bem vindo a mais um vídeo do canal, hoje vamos falar, sobre  MULHER INCENTIVANDO A TIRAR DINHEIRO DOS HOMENS! CORRA PARA AS COLINAS!!  não se esqueça de se inscrever no canal e deixar o like  além disso ative o sininho de notificações para não perder nenhum vídeo. vamos começar;'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Mulher rodada cuidado homem #homem #homens #homemdevalor #homemmoderno #motivação'"},
                {'role': 'assistant', 'content': "1"},
            ],
            'amostra_5': [
                {'role': 'user', 'content': "Text: 'Coisas que o meu marido fala que me deixa sem ar 📕 - AntiOtário 🔱 - Código Masculino🚩 - Red Flags🔞 - Antiporn*🧘 - Lições Perpétuas🔐 - Entrada no Grupo PrivadoEsse é apenas o começo! 📚🗿👇🔗 | https://hotm.art/ComboManual'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'CHEGA NESSA IDADE ELA QUER VIRAR BOA MOÇA | MARI VABO Corte do Episódio: MARI VABO - REDCAST 71!Canal: REDCAST OFICIAL👑 Deixe o seu like e se possível comente!- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 👑 Descubra o poder do Método Alpha: método definitivo para transformar sua vida amorosa.  ⚜️ Aprenda os segredos da conquista, desenvolva sua confiança e se torne o homem que as mulheres desejam. ✅️ Com dicas práticas e estratégias comprovadas, e um passo a passo para se tornar o verdadeiro protagonista de sua jornada amorosa. 🔥 Liberte o seu potencial e conquiste o coração das mulheres com o Método Alpha. Chegou a hora de se destacar! Instagram:   Tik Tok:  #desenvolvimentomasculino #seducao #homemdealtovalor #relacionamento'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'A ROUPA NÃO DEFINE O CARÁTER MAS O CARÁTER DEFINE A ROUPA  #homemdealtovalor #podcast #seducao 📕 Descubra o poder do Método Alpha: método definitivo para transformar sua vida amorosa. Acesse o link abaixo:https://bit.ly/47tJ0bQ- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - Instagram:   Tik Tok:  - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #desenvolvimentomasculino #seducao #homemdealtovalor #relacionamento'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'ELA DIZENDO O QUE OS HOMENS PODEM FAZER NA INTERNET! Olá! seja bem vindo a mais um video do canal, hoje vamos falar, sobre O QUE ELA DIZENDO O QUE OS HOMENS PODEM FAZER NA INTERNET! não se esqueça de se inscrever no canal e deixar o like  além disso ative o sininho de notificações para não perder nenhum video. vamos começar;'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'Por que as Mulheres Estão Ficando Solteiras? A Verdade Chocante CURSO HOMEM BOM DE CAMA:Aprenda como deixar uma Mulher louca na cama (Aulas Explícitas e com modelos reais)👇👇🔞🔞😈😈https://bit.ly/curso-bomdecama✅👉 SERVIÇOS E CURSOS: Acesse o link abaixo 👇👇👇https://papodehomem.my.canva.site/fabricio✅👉 CURSO DE SEDUÇÃO [APRENDA A ARTE DA SEDUÇÃO] Acesse o link abaixo 👇👇👇https://payment.ticto.app/OADA42DF7✅👉RECONQUISTE A SUA EX DE VOLTAAcesse o link abaixo 👇👇👇https://payment.ticto.app/O42378241No vídeo de hoje, Fabrício Acunha vai direto ao ponto sobre um tema que tem se tornado cada vez mais relevante nos dias atuais: mulheres que dizem que não precisam de homens para nada. Esse discurso vem crescendo, especialmente entre as mulheres solteiras acima dos 30 e 40 anos, e tem gerado uma série de consequências nos relacionamentos modernos. Fabrício reflete sobre o aumento da solidão feminina e como muitas mulheres, ao se afirmarem como independentes, acabam se afastando da possibilidade de construir relações saudáveis com os homens.O vídeo aborda o fenômeno das mulheres masculinizadas e com muita energia masculina, que, ao longo dos anos, acabam perdendo a conexão com sua verdadeira essência feminina. Esse tipo de mulher, muitas vezes, desenvolve um comportamento que repele os homens, pois ela assume um papel que, naturalmente, é do homem no relacionamento. O vídeo explora como essa postura faz com que os homens se afastem, pois, ao contrário do que muitas acreditam, os homens não querem se relacionar com mulheres que não reconhecem a necessidade de um parceiro masculino na vida delas.Fabrício também explica que, para que um relacionamento dê certo, a mulher precisa entender que ela precisa de um homem para que o equilíbrio entre as energias femininas e masculinas seja restaurado. A mulher deve se colocar no polo feminino da relação, permitindo que o homem assuma suas funções naturais de liderança e provedor. Quando a mulher tenta ocupar esse espaço e abdica da sua feminilidade, o relacionamento começa a perder sua essência e se torna desequilibrado.A ideia central do vídeo é que os relacionamentos só vão dar certo quando a mulher entender que precisa de homem. Ao reconhecer a importância do masculino em sua vida, ela poderá criar uma dinâmica mais saudável, onde ambos os parceiros podem desempenhar seus papéis de maneira complementar. Esse entendimento é fundamental para que o relacionamento seja duradouro e equilibrado.Se você quer entender melhor como as energias masculina e feminina devem se complementar para um relacionamento dar certo, esse vídeo é para você! Assista, deixe seu like, e não se esqueça de se inscrever no canal para mais conteúdos sobre como melhorar seus relacionamentos e sua vida pessoal.'"},
                {'role': 'assistant', 'content': "1"},
            ],
        },
    },
    'fold_4': {
        'toxicidade': {
            'amostra_1': [
                {'role': 'user', 'content': "Text: '“Ele mandou açaí, eu tava dando pra outro. Comi com o outro” #antiotario #redpill #rafaelaires '"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'ELAS FALAM QUE NÃO PRECISAM DE HOMEM PARA NADA MAS SÓ VIVEM CHORANDO NA INTERNET! Olá! seja bem vindo a mais um vídeo do canal. não se esqueça de se inscrever no canal e deixar o like  além disso ative o sininho de notificações para não perder nenhum vídeo. vamos começar;'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: '🔴 URGENTE 🚨- Brigas, Ciúmes e Controle: Como Elas Tiram a Paz dos Homens! #solteiro #solitude #paz 💥 QUER VIRAR O JOGO? OS CURSOS ESTÃO AQUI ⬇Esse vídeo é um alerta direto aos homens:👉 Se você não prospera, será comandado.👉 Se você não tem poder, será descartado.Se você busca respeito, liberdade e autoridade, esse canal é pra você.Aqui é visão de homem pra homem — sem filtro, sem mimimi.💬 ESCUTA, IRMÃO:Dinheiro é o novo poder.Homem que não aprende a prosperar… vai continuar sendo pisado, traído e ignorado.📚 CURSOS QUE VÃO ABRIR SUA MENTE ⬇✅ Curso 1 – A Regra do Jogo✅ Curso 2 – Segredos dos Investimentos para Iniciantes✅ Curso 3 – Treinamento Bitcoin Black Pill (Renato 38itão)👉 Curso 1 – https://go.hotmart.com/D99197285G👉 Curso 2 – https://go.hotmart.com/U99196870E👉 Curso 3 – https://go.hotmart.com/G93071926D 🔔 Se inscreve no canal, ativa o sininho e compartilha com um homem que precisa acordar.👊 Aqui é realidade, atitude e transformação.👑 Bem-vindo ao canal do Miquéinha.#RelacionamentoTóxico #SolteiroEVivendoMelhor #CiúmesExagerado #HomensAcordem #Relacionamentos #PazMental'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Recebi uma proposta de 5 mil por uma foto do meu pé no PI*. Sou casada, é traição? Aceitei... '"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Você sabe se relacionar bem? | Thiago Schutz #shorts #relacionamento #reflexão #amor #namoro #casal'"},
                {'role': 'assistant', 'content': "0"},
            ],
            'amostra_2': [
                {'role': 'user', 'content': "Text: 'MULHER VAI SER CHIFRUD4 #relacionamento #namoro #casamentonapraia #casal #masculino #macho#seduçao #machoalfa #redpilled #masculinidade #homemdevalor#redpillbrasil #namorosafado #conquistarmulheres #reconquistar #casamento#conquistarmulher #conquistarumamor #fabricioacunha #fabricioacunhamentor #machomacho#amor #cortespodcast #mulher #homem #shorts'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Coisas que me fariam levantar da mesa numa conversa com outras mulheres 🗿 - Antiotario🔱 - Masculino🚩 - Red Flags🔞 - Antiporn*🧘 - Lições Perpétuas🔐 - Entrada no Grupo PrivadoEsse é apenas o começo! 📚🗿👇🔗 | https://hotm.art/ImperioCombo'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'ELAS NÃO ACEITAM QUE AS NOVINHAS GOSTAM DE HOMENS MAIS VELHOS! ACEITA QUE DOÍ MENOSKKKK! Olá! seja bem vindo a mais um vídeo do canal. não se esqueça de se inscrever no canal e deixar o like  além disso ative o sininho de notificações para não perder nenhum vídeo. vamos começar;'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: '2 PASSOS PARA MELHORAR A SUA AUTOESTIMA IMEDIATAMENTE ✅Quer DESTRAVAR a sua vida afetiva definitivamente?👇LINK: https://pay.kiwify.com.br/hEskh2i✅GUIA DA MULHER AUTOCONFIANTEhttps://karinefloriana.com.br/✅E-BOOK: GUIA PRÁTICO PARA AUMENTAR O AMOR-PRÓPRIO E MELHORAR A AUTOESTIMAhttps://pay.kiwify.com.br/SIiljFS📸Instagram➤https://www.instagram.com/karinefloriana/💎Comunidade Mulher Autoconfiante-Receba dicas no Telegramhttps://t.me/+gvhTCF_GHEA2Y2Qx✅ 2 PASSOS PARA MELHORAR A SUA AUTOESTIMA IMEDIATAMENTENeste vídeo vamos falar sobre 2 passos para melhorar a sua autoestima imediatamente. Se você sofre com baixa autoestima e quer aprender como aumentar sua autoestima, esse conteúdo é para você!Descubra como o amor próprio e a valorização pessoal impactam diretamente no seu relacionamento amoroso. Aprenda a se valorizar e construir um relacionamento saudável, atraindo um parceiro que realmente te valorize.🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🔴Compartilhe esse vídeo : 2 PASSOS PARA MELHORAR A SUA AUTOESTIMA IMEDIATAMENTE➤https://youtu.be/VYV_EtAPuS8🚨Videos que também podem te ajudar🚨➤COMO DEIXAR UM HOMEM OBCECADO POR VOCÊ EM 6 PASSOS. AULA 1https://youtu.be/BZfEiq2sMpQ➤COMO CONQUISTAR UM HOMEM COM 4 ATITUDES? FAÇA ELE TE DESEJARhttps://youtu.be/dQBLREVfOmE➤COMO SE TORNAR EXTREMAMENTE ATRAENTE SEM ESFORÇOhttps://youtu.be/kzXwcEa2hOs➤TIPO DE MULHERES SOLTEIRAS QUE OS HOMENS NÃO GOSTAM -MULHERES DESINTERESSANTEShttps://youtu.be/NGwbYWizVjU➤COMO APRENDER A SER AMADA E VALORIZADA POR UM HOMEM RAPIDAMENTEhttps://youtu.be/DchFUqCx9TY➤COMO FAZER UM HOMEM FICAR INTERESSADO POR VOCÊ RAPIDAMENTE- DEIXE ELE APAIXONADOhttps://youtu.be/NsNIPOcIab0➤COMO CHAMAR A ATENÇÃO DE UM HOMEM?  FAÇA DIFERENTEhttps://youtu.be/6wvEfgkfwTI===========================================================🔴 #relacionamento #relacionamentoamoroso #comoconquistarumhomem #conquista 🔻Me Encontre nas Redes Sociais🔻📸Instagram➤https://www.instagram.com/karinefloriana/📸Tiktok➤http://www.tiktok.com/ 🎬YouTube➤https://www.youtube.com/ 📸Linkedin➤www.linkedin.com/in/karinefloriana📸KWAI➤ https://kwai-video.com/u/ /n8XhvCfK'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'O homem que deu uma AULA DE POSTURA diante dessa surtada Uma verdadeira aula de inteligência emocional ao vivo que todo homem possa entender como reagir nessa situação: mulher surta em reality show e acusa outro participante de coisas que ele não fez, e a reação dele foi exemplar.Junte-se ao Conselho, transforme seus hábitos: https://bit.ly/49X2GXoEntre na imersão Vocação e dê sentido a sua vida: https://bit.ly/3qw7PUk📚 Livros que recomendo:Como aprende melhor: https://amzn.to/4cU0VKOProdutividade e pessoas: https://amzn.to/4g4oPWPComo dominar a mente: https://amzn.to/4g5m4EFRelacionamentos: https://amzn.to/4cU1jJgFicções clássicas para homens: https://amzn.to/3AY1u8YFicções para seu filho homem: https://amzn.to/3XoXSpv🎥 Ferramentas De Estudo:Samsung Galaxy Book3: https://amzn.to/3yS6TOyKindle Oasis 8GB: https://amzn.to/3z45l3YCaderno Preto Minimalista ➽ https://amzn.to/3xaI643Bloco Notas Rápidas ➽ https://amzn.to/3AVkbXq🎥 Equipamentos que uso:• Webcam Logitech C920: https://amzn.to/3QGFP7X• Microfone Fifine K669B: https://amzn.to/3QlZHxj• Microfone lapela ECMLV1: https://amzn.to/3SMHhro• Monitor LG 24ML600M: https://amzn.to/3w3omyD• Ring Light 6 Polegadas 36: https://amzn.to/3PrLjm2• HD Externo 1TB USB 3.0 Seagate: https://amzn.to/3A1oRe2• Braço Articulado: https://amzn.to/3bQalxq• Bastão De Luz Brastoy: https://amzn.to/3QIWrfu★ Vitrine Completa de indicações: https://amzn.to/45ziGLW📲 Redes sociais:◼️ Telegram: https://t.me/projetoconselho◼️ Instagram:   / projetoconselho  ◼️ Twitter:   / projetoconselho  ✂️ Canal secundário:    /    ⭐ Quer conversar comigo?Traga suas questões, agende uma consultoria online: ➤ https://bit.ly/3QZ6Cjp◼️ Contato comercial E PIX ◼️➤ contatoprojetoconselho .com--  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --Sobre o canal:Bem-vindo ao Projeto Conselho, espaço de desenvolvimento pessoal masculino e inteligência emocional com o intuito de te ajudar a se tornar um homem melhor. Esse é o meu maior objetivo.'"},
                {'role': 'assistant', 'content': "1"},
            ],
            'amostra_3': [
                {'role': 'user', 'content': "Text: 'E você namoraria uma mulher rodada? Deixe nos comentários! | Qual a sua opinião? 🔥 Se inscreva, comente, curta e compartilhe! 🔥📦| Geração Store ⤵️➡️| Tudo em um só lugar: utilidades, eletrônicos, acessórios e muito mais!https://www.mercadolivre.com.br/social/macedostore📚| Pare de ser feito de Otário! ⤵➡️| hotm.art/geracaoantiotario🚀 O livro que te ensina a se impor, evitar ciladas e nunca mais ser feito de trouxa.'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'A tal friendzone 🗿 - Antiotário🔱 - Código Masculino🚩 - Red Flags🔞 - Antiporn*🧘 - Lições Perpétuas🔐 - Entrada no Grupo PrivadoEsse é apenas o começo! 📚🗿👇🔗 | https://antiotariopro.com/megacombo/'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'COMO FAZER A SUA EX SE ARREPENDER #relacionamento #seducao THIAGO REDPILL E MARI VABO – REAL PODCAST OFICIAL #197 Canal: Real Podcast Oficial- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 👑 Descubra o poder do Método Alpha: método definitivo para transformar sua vida amorosa.  ⚜️ Aprenda os segredos da conquista, desenvolva sua confiança e se torne o homem que as mulheres desejam. ✅️ Com dicas práticas e estratégias comprovadas, e um passo a passo para se tornar o verdadeiro protagonista de sua jornada amorosa. 🔥 Liberte o seu potencial e conquiste o coração das mulheres com o Método Alpha. Chegou a hora de se destacar! Instagram:   Tik Tok:  #desenvolvimentomasculino #seducao #homemdealtovalor'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Isso elas jamais contam😱'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'MULHER COM PAI AUSENTE = VAGABUN DA Clique no link e me siga no novo Instagram  :https://www.instagram.com/sedutorafro22TENHA A MULHER QUE QUISER AUMENTANDO SEU VSM:https://sedutorafro.com.br/aulaodevsm/'"},
                {'role': 'assistant', 'content': "1"},
            ],
            'amostra_4': [
                {'role': 'user', 'content': "Text: 'ELAS PODEM FAZER EXIGÊNCIAS MAS QUANDO OS HOMENS FAZEM ELAS PARTEM PARA PALAVRÕES! #humor #memes Olá! seja bem vindo a mais um video do canal, hoje vamos falar, sobre o porquê que o homem deve malhar e treinar. não se esqueça de se inscrever no canal e deixar o like  além disso ative o sininho de notificações para não perder nenhum video. vamos começar;ELAS PODEM FAZER EXIGÊNCIAS MAS QUANDO OS HOMENS FAZEM ELAS PARTEM PARA PALAVRÕES!'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Mulher de 44 com filhos... 😱'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Essa nova geração está perdida 🗿 - Antiotario🔱 - Masculino🚩 - Red Flags🔞 - Antiporn*🧘 - Lições Perpétuas🔐 - Entrada no Grupo PrivadoEsse é apenas o começo! 📚🗿👇🔗 | https://hotm.art/ImperioCombo'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'PETER EINERD finalmente escancara lacração ANTI-HOMEM no mundo NERD Nerdola? Peter Ei Nerd finalmente critica abertamente a lacração do mundo nerd e abre os olhos dos seguidores sobre a agenda woke.⚜️ Você sabia que temos uma clube de assinatura reunindo o melhor do desenvolvimento pessoal masculino? Entre agora e faça parte do Conselho: https://bit.ly/49X2GXo⚜️ Deseja encontrar o seu propósito e encontrar sentido em tudo o que faz? Conheça as aulas de nosso curso Vocação e através deste conhecimento dê significado a sua vida. → https://bit.ly/3qw7PUk📚 Livros que indico• A História da Virilidade ➽ https://amzn.to/3QJBEs0• O Código dos Homens ➽ https://amzn.to/3JVDGD6• Disciplina é Liberdade ➽ https://amzn.to/3QmTXDz• Responsabilidade Extrema ➽ https://amzn.to/3pgHxRI• Hábitos Atômicos ➽ https://amzn.to/3SMBlyt• A Coragem de Não Agradar ➽ https://amzn.to/3B6IPnY• A Arte de Fazer Acontencer➽ https://amzn.to/3B7sL5D• Como Fazer Amigos E Influenciar Pessoas ➽ https://amzn.to/3qJmpVj• A Educação Da Vontade ➽ https://amzn.to/3DhWQSJ• A Vida Intelectual ➽ https://amzn.to/3QwDrQA• Como Ler Livros ➽ https://amzn.to/3Qpumt2• Como Estudar E Como Aprender ➽ https://amzn.to/3RJmpQE• O Ego É o Seu Inimigo ➽ https://amzn.to/3eC5eCe• A Arte De Viver Melhor ➽ https://amzn.to/3U8ua4y🎥 Ferramentas De Estudo:• Kindle 10a. geração ➽ https://amzn.to/3AknDvB• Caderno Preto Minimalista ➽ https://amzn.to/3xaI643• Bloco Notas Rápidas ➽ https://amzn.to/3AVkbXq🎥 Equipamentos que uso:• Webcam Logitech C920: https://amzn.to/3QGFP7X• Microfone Fifine K669B: https://amzn.to/3QlZHxj• Microfone lapela ECMLV1: https://amzn.to/3SMHhro• Monitor LG 24ML600M: https://amzn.to/3w3omyD• Ring Light 6 Polegadas 36: https://amzn.to/3PrLjm2• HD Externo 1TB USB 3.0 Seagate: https://amzn.to/3A1oRe2• Braço Articulado: https://amzn.to/3bQalxq• Bastão De Luz Brastoy: https://amzn.to/3QIWrfu★ Vitrine Completa de indicações: https://amzn.to/45ziGLW📲 Redes sociais:◼️ Telegram: https://t.me/projetoconselho◼️ Instagram: https://www.instagram.com/projetoconselho◼️ Twitter: https://twitter.com/projetoconselho✂️ Canal secundário: https://www.youtube.com/ ⭐ Quer conversar comigo?Traga suas questões, agende uma consultoria online: ➤ https://bit.ly/3QZ6Cjp◼️ Contato comercial E PIX ◼️➤ contatoprojetoconselho .com--  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --Sobre o canal:Bem-vindo ao Projeto Conselho, espaço de desenvolvimento pessoal masculino e inteligência emocional com o intuito de te ajudar a se tornar um homem melhor. Esse é o meu maior objetivo.'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'O que você faria se a sua namorada tomasse essas atitudes? 1- Adquira o Antiotário 📕2- Domine as lições do nosso livro 3- Faça parte do nosso grupo privado.➡️ https://manualantiotario.com/AntiOtário, livro de desenvolvimento pessoal mais vendido do país. 🫡'"},
                {'role': 'assistant', 'content': "0"},
            ],
            'amostra_5': [
                {'role': 'user', 'content': "Text: 'Minha mina terminou e tá seguindo vários mlks no insta #VSM #relacionamentosEntre para o Super Aulão de VSM: http://bit.ly/3RUC29n Neste vídeo, Gabriel Amorim dá as melhores dicas para você esquecer sua ex-namorada e virar a página de uma vez por todas.'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'DIVÓRCIO APÓS 50 ANOS! MULHERES SÃO AS MAIORES CAUSADORAS! E SIM, UMA NOVA TENTATIVA VAI FRACASSAR Aproveito que o tema do dicórcio cinza veio à tona novamente para mandar verdades mais pesadas do que chumbo. Mais uma vez o sistema tenta achar mil desculpas para esse fenômeno e não dizem o real causador de todos os problemas de relacionamento que aocntecem na nossa sociedade: o feminismo.O feminismoo veio como um vírus que atacou homens, mas principalmente mulheres. Esse vírus afetou o modo de pensar das mulheres de uma forma que elas estão fora da normalidade. Eu sou o mensageiro da verdade. Tudo aquilo que estava escondido veio à tona porque eu me propus a fazer esse trabalho. Vou tirar da ilusão e do autoengano o máximo de homens possível e se for preciso direi palavras pesadas e difíceis de assimilar.No caso do divórcio cinza, trago a realidade desse fenômeno e aponto o culpado. https://www.diariodepernambuco.com.br/colunas/diariomulher/2025/01/divorcio-cinza-um-fenomeno-crescente-entre-os-maiores-de-50-anos.html#relacionamento #casamento #divorcio'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Pistolinha #hackeandoamatrix Hoje no hackeando a matrix tem o pai da atriz Andressa Urach pedindo ajuda de sua filha pra ajuda-lo a entrar no mundo da produção audiovisual igual Andressa Urach, ela vazou o audio em que o pai  pede essa oportunidade e fez piada com isso.        ⭐Seja membro para ter acesso a vídeos exclusívos por apenas $2,99 🤏     https://www.youtube.com/channel/UCDrGZXd8k06ifz0-gccIn0w/join#hackeandoamatrix #morenotatuado #homemdealtovalor-----------------------------------------------------------------------------------------------------------Esse é um canal de opinião, onde analiso dinâmicas sociais entre homens e mulheres do ponto de vista de um homem.Meu compromisso é dar minha contribuição sincera e respeitosa sobre assuntos da atualidade que interessam ao homem de alto valor.Quando o homem começa a seguir o próprio caminho aprende que precisa se manter bem informado.💊DEIXE O LIKE!                       💊SE INSCREVA!             👉Compartilhe esse vídeo com alguém que vc gosta:Vazou Áudio do Pai da Andressa Urach Pede Pra Gravar COM ELAhttps://youtu.be/b7MlWYCfPnY'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'NETFLIX FAZ UMA MINISSÉRIE HORRENDA COM LACRAÇÃO DO COMEÇO AO FIM! ALVO: HOMEM BRANCO HETEROSSEXUAL Não vou me estender muito em mais explicações porque o vídeo explica o que se é preciso entender. Série da Netflix deixa, para o espectador atento, várias lacrações sútis (para os iludidos, não). Uma clara demonstração de uma sociedade autofágica. O próprio ocidente se autodestrói em uma sanha louca e revolucionária que para os leigos é impossível de entender.#minissérie #adolescencia #netflixseries'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: '“Casada, envolvi com outro Que não tem nem uma bike. Não me quer e eu só choro SOS” 🍷🗿 Senhores…1. Obtenha o Mega Combo 📕2. Solicite sua entrada no grupo privado 🔐3. Desfrute de toda a sabedoria 🍷🗿Te espero lá ⤵🔗 - https://hotm.art/ComboManual'"},
                {'role': 'assistant', 'content': "0"},
            ],
        },
        'misoginia': {
            'amostra_1': [
                {'role': 'user', 'content': "Text: 'Primeira Psicóloga Especialista Em Leitada e Ghost Do Brasil Hoje no hackeando a matrix tem uma especialista formada em psicologia falando a respeito do ghosting ela da dicas para compreender e superar esse problema.    ⭐Seja membro para ter acesso a vídeos exclusívos por apenas $2,99 🤏     https://www.youtube.com/channel/UCDrGZXd8k06ifz0-gccIn0w/join#hackeandoamatrix #morenotatuado #homemdealtovalor-----------------------------------------------------------------------------------------------------------Esse é um canal de opinião, onde analiso dinâmicas sociais entre homens e mulheres do ponto de vista de um homem.Meu compromisso é dar minha contribuição sincera e respeitosa sobre assuntos da atualidade que interessam ao homem de alto valor.Quando o homem começa a seguir o próprio caminho aprende que precisa se manter bem informado.💊DEIXE O LIKE!                       💊SE INSCREVA!             👉Compartilhe esse vídeo com alguém que vc gosta:Psicóloga Especialista Em Leitada e Ghosthttps://youtu.be/4nZkWC03PM4'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'É MELHOR O MIQUEINHA ACREDITAR NO PAI NOEL DO QUE NA DEUSA RARA! Olá! seja bem vindo a mais um video do canal, hoje vamos falar, sobre É MELHOR O MIQUEINHA ACREDITAR NO PAI NOEL DO QUE NA DEUSA RARA!não se esqueça de se inscrever no canal e deixar o like  além disso ative o sininho de notificações para não perder nenhum video. vamos começar;'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'VEM AÍ O ESTELIONATO SENTIMENTAL! O SISTEMA AGORA AGE PARA ALCANÇAR OS HOMENS FORA DE RELACIONAMENTO Não passaria despercebido pelo sistema feminista que se os homens se absterem de relacionamentos sérios, as chances de se darem mal caem absurdamente. Por isso, há projeto em andamento que busca alcançar os homens que não estão em compromisso sério através do absurdo estelionato sentimental!É algo mais difícil de se provar, mas colocará mais um empecilho para os conquistadores de plantão. Aquele que insistir em ter namoradinhas e ficantes fixas começarão a correr perigo. Apesar da lei, em tese, valer para os homens, todos nós sabemos que é dirigido para benefício das mulheres.PL 3023/2024#relacionamento #namoro'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: '“A narcisista é capaz de deixar um homem em situação de pobreza financeira” 📕 - AntiOtário 🔱 - Código Masculino🚩 - Red Flags🔞 - Antiporn*🧘 - Lições Perpétuas🔐 - Entrada no Grupo PrivadoEsse é apenas o começo! 📚🗿👇🔗 | https://hotm.art/ComboManual'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Dá pra manter o fogo depois de casado? | Thiago Schutz #shorts #relacionamento #fogo #reflexão'"},
                {'role': 'assistant', 'content': "0"},
            ],
            'amostra_2': [
                {'role': 'user', 'content': "Text: 'A digníssima do Tinder #homem #homens #homemdevalor #homemmoderno  #motivacao'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Verdades sobre namorar um nerd 1- Adquira o Antiotário 📕2- Domine as lições do nosso livro 3- Faça parte do nosso grupo privado.➡️ https://manualantiotario.com/AntiOtário, livro de desenvolvimento pessoal mais vendido do país. 🫡'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'MULHER EXIGENTE FICA SOZINHA #relacionamento #namoro #casamentonapraia #casal #masculino #macho#seduçao #machoalfa #redpilled #masculinidade #homemdevalor#redpillbrasil #namorosafado #conquistarmulheres #reconquistar #casamento#conquistarmulher #conquistarumamor #fabricioacunha #fabricioacunhamentor #machomacho#amor #cortespodcast #mulher #homem #shorts'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'E aí, você vive na Disney? | Qual a sua opinião? #rafaelaires  #antiotario  #redpillbrasil'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'Mulher, quanto mais preservada, mais será amada! | Qual a sua opinião? #rafaelaires #antiotario #redpillbrasil🔥 Pare de ser feito de Otário! 🔥📕 Manual AntiOtário✅ Evitar manipulações e jogos emocionais✅ Se posicionar com inteligência nos relacionamentos✅ Torne-se a melhor versão de si mesmo📚 Adquira o seu agora! ⤵➡️| hotm.art/geracaoantiotario🚀 O livro que te ensina a se impor, evitar ciladas e nunca mais ser feito de trouxa. '"},
                {'role': 'assistant', 'content': "1"},
            ],
            'amostra_3': [
                {'role': 'user', 'content': "Text: 'RELATO EMOCIONANTE DE MSOL! FIQUEI ATÉ EMOCIONADO! Olá! seja bem vindo a mais um video do canal, hoje vamos falar, sobre RELATO EMOCIONANTE DE MSOL! FIQUEI ATÉ EMOCIONADO! não se esqueça de se inscrever no canal e deixar o like  além disso ative o sininho de notificações para não perder nenhum video. vamos começar;'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'PURPLE PILL, A PÍLULA DO ABSURDO ► Alphalife+: https://alphalifeplus.com/?utm_source=youtube► Lover Academy: https://loveracademy.com.br/?utm_source=youtube► Close Friends: https://copinioficial.com/?utm_source=youtubeExplore os mistérios e paradoxos da existência humana com a teoria do absurdismo de Albert Camus neste vídeo imperdível! 🎭📚 Descubra como Camus desafia as noções tradicionais de sentido e propósito da vida, propondo uma visão única e provocativa sobre a condição humana. 💡💭 Prepare-se para uma jornada intelectual estimulante, enquanto mergulhamos nas ideias profundas e intrigantes deste renomado filósofo francês. Desde o confronto com o absurdo até a busca pela revolta e liberdade, este vídeo oferece uma perspectiva nova e instigante sobre o significado da existência. Assista agora e embarque nessa viagem filosófica rumo ao âmago do absurdo! ► Instagram: https://www.instagram.com/copini_alphalife/► Contato Suporte para alunos: alphalifesuporte .comEmail comercial: alphalifecopini .com#Absurdismo #AlbertCamus #FilosofiaExistencial #CondiçãoHumana #JornadaIntelectual #ProvocaçãoFilosófica'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'a culpa é dos hom3ns - E EU COM ISSO?? #mentalidadeblack #mestredablack #redpill #desenvolvimentopessoal #masculinidade 1️⃣ Canal do telegramhttps://t.me/+bQTSMHl9Z1o4NDAx2️⃣ COMBO 20 CURSOShttps://pay.kiwify.com.br/jQ9YQDv?afi...3️⃣ Secretaria da escola(dúvidas referente ao curso)https://wa.me/5522992232841ESSE CANAL trata da perspectiva MASCULINA do mundo moderno e DESENVOLVIMENTO PESSOAL opiniões sarcásticas e humoradaspeço respeito nos comentários'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Três tipos de comportamento que eu considero falta de respeito dentro de um relacionamento 🗿 - Antiotario🔱 - Masculino🚩 - Red Flags🔞 - Antiporn*🧘 - Lições Perpétuas🔐 - Entrada no Grupo PrivadoEsse é apenas o começo! 📚🗿👇🔗 | https://hotm.art/ImperioCombo'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'Por isso que eles se relacionam com esse tipo de mulher | Thiago Schutz #shorts #reflexão #amor #fyp'"},
                {'role': 'assistant', 'content': "1"},
            ],
            'amostra_4': [
                {'role': 'user', 'content': "Text: 'COMO FAZER QUALQUER MULHER TER MEDO DE TE PERDER (fatos científicos) AUMENTE SEU VSM E TENHA A MULHER QUE QUISER:https://sedutorafro.com.br/aulaodevsm/'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Depois de rodar e rodar a mente bagunça e o culpado será quem é o atual, cuidado homem #homem Muitas ficarão assim depois de rodar e rodar e jogarão a culpa no parceiro atual cuidado Homem Nunca mais seja enganado, saiba todos os truques usados por elas neste link 👉 hotm.art/Antitrouxa 👈#homem #homens #motivacao #homemmoderno  #homemdevalor'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: '“Talvez o seu casamento esteja indo por água abaixo porque você está militando demais” 🗿🍷 📕 - Antiotario🔱 - Código Masculino🚩 - Red Flags🔞 - Antiporn*🧘 - Lições Perpétuas🔐 - Entrada no Grupo PrivadoEsse é apenas o começo! 📚🗿👇🔗 | https://hotm.art/ComboManual'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'Bizarrices de Débora Secco #shorts'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'ESSE É O PLANO DOS GLOBALISTAS'"},
                {'role': 'assistant', 'content': "0"},
            ],
            'amostra_5': [
                {'role': 'user', 'content': "Text: 'MIQUEINHA CORNO CONFORMADO! #memes #humor #amelhorredpillvemdelas #comedia seja bem vindo a mais um vídeo do canal Não se esqueça inscrever no CANAL e deixar o like  além disso ative o sininho de notificações para não perder nenhum vídeo.MIQUEINHA CORNO CONFORMADO!'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'O QUE A MULHER BUSCA EM UM RELACIONAMENTO AMOROSO? #relacionamento #emocional #segurança #amor ✅O QUE A MULHER BUSCA EM UM RELACIONAMENTO AMOROSO?Neste vídeo eu vou te explicar sobre o que a mulher busca em um relacionamento amoroso..Aqui você também irá compreender sobre relacionamento amoroso, sobre o que é mais importante no relacionamento, sobre amor, sobre relacionamentos amorosos e sobre namoro.🔴GUIA DA MULHER AUTOCONFIANTE🔴https://karinefloriana.com.br/💎Receba DICAS no Telegramhttps://t.me/+gvhTCF_GHEA2Y2Qx🔴E-BOOK: GUIA PRÁTICO PARA AUMENTAR O AMOR-PRÓPRIO E MELHORAR A AUTOESTIMAhttps://pay.hotmart.com/C83942882P🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🔴Compartilhe esse vídeo : ➤O QUE A MULHER BUSCA EM UM RELACIONAMENTO AMOROSO?https://youtube.com/shorts/AcWcigUqcfs🚨Videos que também podem te ajudar🚨➤Por que os homens esfriam ou se afastam de você? Aula 1https://youtu.be/_qNB7lmWWp0➤Por que os homens esfriam ou se afastam de você? Aula 2https://youtu.be/zzT6vXgU8V0➤COMO ME PREPARAR PARA AMAR E SER AMADAhttps://youtu.be/Vj_y7N2BCQk➤COMO SER FELIZ NO AMOR? PARTE 1https://youtu.be/KNv-l5899KU➤COMO FAZER O HOMEM SE APAIXONAR POR VOCÊ.  AULA 1/ A MENTE MASCULINA.https://youtu.be/8EnP_mLQeSc➤COMO FAZER O HOMEM SE APAIXONAR POR VOCÊ. AULA 2/ OS MAIORES ERROS QUE AS MULHERES COMETEMhttps://youtu.be/8EnP_mLQeSc➤COMO FAZER O HOMEM SE APAIXONAR POR VOCÊ .AULA 3 -DANÇA DA DUALIDADE FEMININAhttps://youtu.be/I7mwxZBNIdM➤COMO FAZER O HOMEM SE APAIXONAR POR VOCÊ. AULA -4 AÇÃOhttps://youtu.be/diEfQEneFMc===========================================================🔴 #relacionamento #amor #linguagensdoamor #amar #relacao #afetiva 🔻Me Encontre nas Redes Sociais🔻📸Instagram➤https://www.instagram.com/karinefloriana/📸Tiktok➤http://www.tiktok.com/ 🎬YouTube➤https://www.youtube.com/ 📸Linkedin➤www.linkedin.com/in/karinefloriana📸KWAI➤ https://kwai-video.com/u/ /n8XhvCfK'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Mulheres Israelitas 🎯😍'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'AS MULHERES QUARENTONAS SÃO O NOVO PRÊMIO? 💪 Use o cupom (CONSELHO) na Growth: https://www.gsuplementos.com.br/𖣠 Faça parte do Alto Conselho, transforme seus hábitos: https://altoconselho.com𖣠 Conheça a imersão Vocação, dê sentido a sua vida: https://bit.ly/3qw7PUk📚 Livros que recomendo: https://amzn.to/3PNAUDX🎥 Equipamentos que uso: https://amzn.to/3PGT36m✂️ Canal secundário: https://www.youtube.com/ ☕︎ Quer conversar comigo?Traga suas questões, agende uma consultoria: https://bit.ly/3QZ6Cjp◼️ Contato comercial◼️➤ contatoprojetoconselho .com'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Um indício de traição difícil de perceber!?'"},
                {'role': 'assistant', 'content': "0"},
            ],
        },
    },
    'fold_5': {
        'toxicidade': {
            'amostra_1': [
                {'role': 'user', 'content': "Text: 'UBER casado DESABAFOU no carro: está IMPOSSÍVEL desse jeito! ⚜️ Você sabia que temos uma clube de assinatura reunindo o melhor do desenvolvimento pessoal masculino? Entre agora e faça parte do Conselho: https://bit.ly/49X2GXo⚜️ Deseja encontrar o seu propósito e encontrar sentido em tudo o que faz? Conheça as aulas de nosso curso Vocação e através deste conhecimento dê significado a sua vida. → https://bit.ly/3qw7PUk📚 Livros que indico• A História da Virilidade ➽ https://amzn.to/3QJBEs0• O Código dos Homens ➽ https://amzn.to/3JVDGD6• Disciplina é Liberdade ➽ https://amzn.to/3QmTXDz• Responsabilidade Extrema ➽ https://amzn.to/3pgHxRI• Hábitos Atômicos ➽ https://amzn.to/3SMBlyt• A Coragem de Não Agradar ➽ https://amzn.to/3B6IPnY• A Arte de Fazer Acontencer➽ https://amzn.to/3B7sL5D• Como Fazer Amigos E Influenciar Pessoas ➽ https://amzn.to/3qJmpVj• A Educação Da Vontade ➽ https://amzn.to/3DhWQSJ• A Vida Intelectual ➽ https://amzn.to/3QwDrQA• Como Ler Livros ➽ https://amzn.to/3Qpumt2• Como Estudar E Como Aprender ➽ https://amzn.to/3RJmpQE• O Ego É o Seu Inimigo ➽ https://amzn.to/3eC5eCe• A Arte De Viver Melhor ➽ https://amzn.to/3U8ua4y🎥 Ferramentas De Estudo:• Kindle 10a. geração ➽ https://amzn.to/3AknDvB• Caderno Preto Minimalista ➽ https://amzn.to/3xaI643• Bloco Notas Rápidas ➽ https://amzn.to/3AVkbXq🎥 Equipamentos que uso:• Webcam Logitech C920: https://amzn.to/3QGFP7X• Microfone Fifine K669B: https://amzn.to/3QlZHxj• Microfone lapela ECMLV1: https://amzn.to/3SMHhro• Monitor LG 24ML600M: https://amzn.to/3w3omyD• Ring Light 6 Polegadas 36: https://amzn.to/3PrLjm2• HD Externo 1TB USB 3.0 Seagate: https://amzn.to/3A1oRe2• Braço Articulado: https://amzn.to/3bQalxq• Bastão De Luz Brastoy: https://amzn.to/3QIWrfu★ Vitrine Completa de indicações: https://amzn.to/45ziGLW📲 Redes sociais:◼️ Telegram: https://t.me/projetoconselho◼️ Instagram: https://www.instagram.com/projetoconselho◼️ Twitter: https://twitter.com/projetoconselho✂️ Canal secundário: https://www.youtube.com/ ⭐ Quer conversar comigo?Traga suas questões, agende uma consultoria online: ➤ https://bit.ly/3QZ6Cjp◼️ Contato comercial E PIX ◼️➤ contatoprojetoconselho .com--  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --Sobre o canal:Bem-vindo ao Projeto Conselho, espaço de desenvolvimento pessoal masculino e inteligência emocional com o intuito de te ajudar a se tornar um homem melhor. Esse é o meu maior objetivo.'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'SER FRACASSADO E TRISTE É O NOVO NORMAL O novo normal é sobre se sentir bem ser triste e fracassado.► BLACKBOX (acesso a tudo por R$59,99/mês) https://alphalifeplus.com/black-box-2024/► Instagram: https://www.instagram.com/copini_alphalife/► Contato Suporte para alunos: alphalifesuporte .comEmail comercial: alphalifecopini .com'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'Se for Chad não existe exigências #homem #homens #homemdevalor #homemmoderno #motivação'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: '4 coisas que o homem nunca deve aceitar numa mulher #VSM #relacionamentosEntre para o Super Aulão de VSM: http://bit.ly/3RUC29n Neste vídeo, Gabriel Amorim dá as melhores dicas para você esquecer sua ex-namorada e virar a página de uma vez por todas.'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Homem cuidado isso pode ser interpretado como assédio 🗿 - Antiotario🔱 - Masculino🚩 - Red Flags🔞 - Antiporn*🧘 - Lições Perpétuas🔐 - Entrada no Grupo PrivadoEsse é apenas o começo! 📚🗿👇🔗 | https://hotm.art/ImperioCombo'"},
                {'role': 'assistant', 'content': "1"},
            ],
            'amostra_2': [
                {'role': 'user', 'content': "Text: 'Existem 2 tipos de mulher #seducao #mulherengo #masculinidade #namoro #conquista #mulheres'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'O PERIGOSO TEMA DA BLACKPILL DA APARÊNCIA! UMA REFLEXÃO DEFINITIVA PARA HOMENS AFASTAREM-SE DISSO! Há muito tempo eu devia um vídeo sobre o tema da blackpill da aparência. Não considerava um assunto com grande relevância para meu intento em combater o sistema feminista. Mas, vejo que se tornou um assunto perigoso, principalmente para homens jovens.Produtores se aproveitam de uma necessidade dos homens jovens para levá-los a considerar que somente a aparência importa. Todo o resto pode ser jogado na lata de lixo. Isso pode levar esse jovens a entrarem em uma espiral de desânimo, depressão, porque será inculcado na mente deles que a única coisa que importa é ser bonito para conquistar mulheres. Esses produtores são plagiadores de canais estrangeiros sobre o tema e suas aparências que são escondidas, quando reveladas, mostram que são homens abaixo da média. São figuras sofríveis, o retrato do brasileiro de periferia. São sujeitos que passam despercebidos pelas ruas brasileiras. Por isso fiz esse vídeo para ajudar os homens jovens, principalmente, a abandonarem esse conteúdo. Suas vidas valem mais do que aparência. Nós somos homens e não Zé-Ruelas com braços de mosquito que querem decidir o que é importante e o que não é.#relacionamento #casamento #dinheiro'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'O QUE FEZ VOCÊ SE CONECTAR NO INÍCIO ? Desvendando a Mente Feminina: A Sabedoria de Mari Vabo no Hey Folks - Mari Vabo #47Episódio completo: https://www.youtube.com/watch?v=HLef2f-KmqoCanal: Hey Folks - Podcasthttps://www.youtube.com/ Convidada: Mari Vabohttps://www.youtube.com/ - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 👑 Deixe o seu like e comentário!📕 Aprenda como conquistar qualquer mulher. Acesse o link: https://bit.ly/47tJ0bQ - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 👑 Deixe o seu like e comentário!- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - Instagram:   Tik Tok:   - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #desenvolvimentomasculino #seducao #homemdealtovalor #relacionamento #psicologiacomportamental #psicologaderelacionamento #inteligenciaemocional #linguagemcorporal'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'MULHER RODOU O CARROSSEL AGORA QUER UM  HOMEM CRISTÃO PARA CASAR! #memes #humor#comedia seja bem vindo a mais um vídeo do canal Não se esqueça inscrever no CANAL e deixar o like  além disso ative o sininho de notificações para não perder nenhum vídeo.MULHER RODOU O CARROSSEL AGORA QUER UM  HOMEM CRISTÃO PARA CASAR! #memes #humor#comedia'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Agora ficou claro? #serjaodosfoguetes #sergiosacani'"},
                {'role': 'assistant', 'content': "0"},
            ],
            'amostra_3': [
                {'role': 'user', 'content': "Text: 'Relacionamento hoje em dia está cada vez mais difícil por conta de algumas “mulheres”. #Redpill #sigma #evolução #homem #homemdevalor #masculinidade #valores #motivação #react #respeito #humildade #principios #evoluir #shorts'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'como enriquecer sendo POBRE. [direto ao ponto] ► Capital Digital: https://matheuscopini.com/Aqui vão algumas verdades sobre esriquecer vindo de baixo, sem ajuda de ninguém. E também algumas fatores sobre psicologia do consumo, que faz qualquer pessoa enriquecer.► Capital Digital: https://alphalifeplus.com/?utm_source=youtube► Estoicismo diário: https://matheuscopini.com/estoicismodiario► Newsletter: Capital Society: https://capitalsociety.substack.com/subscribe#desenvolvimentopessoal #marketingdigital #criadordeconteudo Me encontre nas outras redes► Instagram   https://www.instagram.com/copini_alphalife/► Tiktok https://www.tiktok.com/    ► Spotify https://open.spotify.com/show/4YHh4JPSMSJzcR8HxsIFfEContato:Suporte para alunos: alphalifesuporte .comEmail comercial: alphalifecopini .comWhatsapp comercial: 54 81228453'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'Se o que você busca é sexo intenso, frequência alta e adrenalina, não case! | Qual a sua opinião? #rafaelaires #antiotario #redpillbrasil🔥 Pare de ser feito de Otário! 🔥📕 Manual AntiOtário✅ Evitar manipulações e jogos emocionais✅ Se posicionar com inteligência nos relacionamentos✅ Torne-se a melhor versão de si mesmo📚 Adquira o seu agora! ⤵➡️| hotm.art/geracaoantiotario🚀 O livro que te ensina a se impor, evitar ciladas e nunca mais ser feito de trouxa.'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'SE É FEIO PARA A MULHER IMAGINE PARA O HOMEM  #homemdealtovalor #comportamentomasculino #seducao 📕 Descubra o poder do Método Alpha: método definitivo para transformar sua vida amorosa. Acesse o link abaixo:https://bit.ly/47tJ0bQ- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - Instagram:   Tik Tok:  - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #desenvolvimentomasculino #seducao #homemdealtovalor #relacionamento'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: '“Se você mora sozinho há 3 anos, sinto lhe informar mas você é um viciado em tranquilidade” 🍷🗿 📕 - Antiotario🔱 - Código Masculino🚩 - Red Flags🔞 - Antiporn*🧘 - Lições Perpétuas🔐 - Entrada no Grupo PrivadoEsse é apenas o começo! 📚🗿👇🔗 | https://hotm.art/ComboManual'"},
                {'role': 'assistant', 'content': "0"},
            ],
            'amostra_4': [
                {'role': 'user', 'content': "Text: 'VILELA foi CHUT4DO pela EX 10 DE 10 #mentalidadeblack #mestredablack #redpill #desenvolvimentopessoal #masculinidade CURSO BITCOIN 10 REAIS:mercado pago:https://mpago.la/12Cyu4ochave PIX para pagamento no mercado pago:22992232841PIX concluido,é só chamar no whatsapp e informar o comprovante:https://wa.me/5522992232841COMBO JOVEM EMPRESÁRIO:https://pay.kiwify.com.br/jQ9YQDv?afid=zh8vwn7LESSE CANAL trata da perspectiva MASCULINA do mundo moderno e DESENVOLVIMENTO PESSOAL opiniões sarcásticas e humoradaspeço respeito nos comentários'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'Mulher gosta de mala sem alça 🤦🏽'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Essas deram aulas! | Qual a sua opinião? #rafaelaires  #antiotario  #redpillbrasil'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'FEMINISTA termina sem motivo e CHORA depois #mentalidadeblack #mestredablack #redpill #desenvolvimentopessoal #masculinidade ESSE CANAL trata da perspectiva MASCULINA do mundo moderno e DESENVOLVIMENTO PESSOAL opiniões sarcásticas e humoradaspeço respeito nos comentários'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Seu marido é assim? #homem #homens #homemdevalor #homemmoderno #motivacao Se ela falar isso abre o olho homem Nunca mais seja enganado, saiba todos os truques usados por elas neste link 👉 hotm.art/Antitrouxa 👈#homem #homens #motivacao #homemmoderno  #homemdevalor'"},
                {'role': 'assistant', 'content': "1"},
            ],
            'amostra_5': [
                {'role': 'user', 'content': "Text: 'ELAS ESTÃO DESESPERADAS POIS NÃO TEM MAIS TEMPO A PERDER O INVERNO ESTÁ CHEGANDO PARA ELAS! seja bem vindo a mais um vídeo do canal,  ATENÇÃO hoje vamos ver!!ELAS ESTÃO DESESPERADAS POIS NÃO TEM MAIS TEMPO A PERDER O INVERNO ESTÁ CHEGANDO PARA ELAS! Não se esqueça inscrever no CANAL e deixar o like  além disso ative o sininho de notificações para não perder nenhum vídeo. vamos começar'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'RED FLAGS! #memes #humor #relaciomento #comedia Olá! seja bem vindo a mais um video do canal, hoje vamos falar, sobre o porquê que o homem deve malhar e treinar. não se esqueça de se inscrever no canal e deixar o like  além disso ative o sininho de notificações para não perder nenhum video. vamos começar;RED FLAGS!'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'PURPLE PILL, A PÍLULA DO ABSURDO ► Alphalife+: https://alphalifeplus.com/?utm_source=youtube► Lover Academy: https://loveracademy.com.br/?utm_source=youtube► Close Friends: https://copinioficial.com/?utm_source=youtubeExplore os mistérios e paradoxos da existência humana com a teoria do absurdismo de Albert Camus neste vídeo imperdível! 🎭📚 Descubra como Camus desafia as noções tradicionais de sentido e propósito da vida, propondo uma visão única e provocativa sobre a condição humana. 💡💭 Prepare-se para uma jornada intelectual estimulante, enquanto mergulhamos nas ideias profundas e intrigantes deste renomado filósofo francês. Desde o confronto com o absurdo até a busca pela revolta e liberdade, este vídeo oferece uma perspectiva nova e instigante sobre o significado da existência. Assista agora e embarque nessa viagem filosófica rumo ao âmago do absurdo! ► Instagram: https://www.instagram.com/copini_alphalife/► Contato Suporte para alunos: alphalifesuporte .comEmail comercial: alphalifecopini .com#Absurdismo #AlbertCamus #FilosofiaExistencial #CondiçãoHumana #JornadaIntelectual #ProvocaçãoFilosófica'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'A TRAIÇÃO FEMININA NÃO PODE SER ESQUECIDA! NEM PELOS HOMENS DESPERTOS E NEM PELOS BOIS DOMADOS Há muito tempo atrás já fiz mais de um video falando sobre a questão de que as mulheres já estavam traindo mais do que os homens. Isso faz parte do profano feminino, exacerbado pelo feminismo e que não pode se perder na memória. A inistência em temas como a traição feminina deve ser sempre relembrada para que os homens despertos não venham a esquecer e também serve como mais um exemplo para os críticos raivosos que jogam a culpa somente nos homens pelo decréscimo nos relacionamentos.Logicamente, esse monte de bois de canga não vão rebater o que eu digo. O que eles só sabem fazer é criticar os homens (sabem que não dá em nada) e deixar de fora as maiores culpadas pela derrocada dos casamentos que são as mulheres.Não importa se o feminismo abriu as portas do profano feminino, o que importa é que as mulheres abraçaram essa nova modalidade de comportamento e isso precisa ser jogado na cara desses guampas secas.  https://deolhonacidade.net/detetive-em-feira-relata-que-as-mulheres-traem-mais-que-os-homens/#relacionamento #casamento #crianças'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'SE A MULHER GOSTA DISSO NEM CHEGUE PERTO | MARI VABO Corte do Episódio: MARI VABO - REDCAST 71!Canal: REDCAST OFICIAL👑 Deixe o seu like e se possível comente!- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 👑 Descubra o poder do Método Alpha: método definitivo para transformar sua vida amorosa.  ⚜️ Aprenda os segredos da conquista, desenvolva sua confiança e se torne o homem que as mulheres desejam. ✅️ Com dicas práticas e estratégias comprovadas, e um passo a passo para se tornar o verdadeiro protagonista de sua jornada amorosa. 🔥 Liberte o seu potencial e conquiste o coração das mulheres com o Método Alpha. Chegou a hora de se destacar! Instagram:   Tik Tok:  #desenvolvimentomasculino #seducao #homemdealtovalor #relacionamento'"},
                {'role': 'assistant', 'content': "1"},
            ],
        },
        'misoginia': {
            'amostra_1': [
                {'role': 'user', 'content': "Text: 'o DRAMA dos RELACIONAMENTOS depois dos 30 ⚜️ Você sabia que temos uma clube de assinatura reunindo o melhor do desenvolvimento pessoal masculino? Entre agora e faça parte do Conselho: https://bit.ly/49X2GXo⚜️ Deseja encontrar o seu propósito e encontrar sentido em tudo o que faz? Conheça as aulas de nosso curso Vocação e através deste conhecimento dê significado a sua vida. → https://bit.ly/3qw7PUk📚 Livros que indico• A História da Virilidade ➽ https://amzn.to/3QJBEs0• O Código dos Homens ➽ https://amzn.to/3JVDGD6• Disciplina é Liberdade ➽ https://amzn.to/3QmTXDz• Responsabilidade Extrema ➽ https://amzn.to/3pgHxRI• Hábitos Atômicos ➽ https://amzn.to/3SMBlyt• A Coragem de Não Agradar ➽ https://amzn.to/3B6IPnY• A Arte de Fazer Acontencer➽ https://amzn.to/3B7sL5D• Como Fazer Amigos E Influenciar Pessoas ➽ https://amzn.to/3qJmpVj• A Educação Da Vontade ➽ https://amzn.to/3DhWQSJ• A Vida Intelectual ➽ https://amzn.to/3QwDrQA• Como Ler Livros ➽ https://amzn.to/3Qpumt2• Como Estudar E Como Aprender ➽ https://amzn.to/3RJmpQE• O Ego É o Seu Inimigo ➽ https://amzn.to/3eC5eCe• A Arte De Viver Melhor ➽ https://amzn.to/3U8ua4y🎥 Ferramentas De Estudo:• Kindle 10a. geração ➽ https://amzn.to/3AknDvB• Caderno Preto Minimalista ➽ https://amzn.to/3xaI643• Bloco Notas Rápidas ➽ https://amzn.to/3AVkbXq🎥 Equipamentos que uso:• Webcam Logitech C920: https://amzn.to/3QGFP7X• Microfone Fifine K669B: https://amzn.to/3QlZHxj• Microfone lapela ECMLV1: https://amzn.to/3SMHhro• Monitor LG 24ML600M: https://amzn.to/3w3omyD• Ring Light 6 Polegadas 36: https://amzn.to/3PrLjm2• HD Externo 1TB USB 3.0 Seagate: https://amzn.to/3A1oRe2• Braço Articulado: https://amzn.to/3bQalxq• Bastão De Luz Brastoy: https://amzn.to/3QIWrfu★ Vitrine Completa de indicações: https://amzn.to/45ziGLW📲 Redes sociais:◼️ Telegram: https://t.me/projetoconselho◼️ Instagram: https://www.instagram.com/projetoconselho◼️ Twitter: https://twitter.com/projetoconselho✂️ Canal secundário: https://www.youtube.com/ ⭐ Quer conversar comigo?Traga suas questões, agende uma consultoria online: ➤ https://bit.ly/3QZ6Cjp◼️ Contato comercial E PIX ◼️➤ contatoprojetoconselho .com--  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --Sobre o canal:Bem-vindo ao Projeto Conselho, espaço de desenvolvimento pessoal masculino e inteligência emocional com o intuito de te ajudar a se tornar um homem melhor. Esse é o meu maior objetivo.'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'APRENDA A GERAR ATRAÇÃO NAS MULHERES #relacionamento #namoro #casamentonapraia #casal #masculino #macho#seduçao #machoalfa #redpilled #masculinidade #homemdevalor#redpillbrasil #namorosafado #conquistarmulheres #reconquistar #casamento#conquistarmulher #conquistarumamor #fabricioacunha #fabricioacunhamentor #machomacho#amor #cortespodcast #mulher #homem'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'MULHER QUE SE MOSTRA DEMAIS QUER DIZER QUE É CARENTE #relacionamento #namoro #casamentonapraia #relacionamento #namoro #casamentonapraia #casal #masculino #macho#seduçao #machoalfa #redpilled #masculinidade #homemdevalor#redpillbrasil #namorosafado #conquistarmulheres #reconquistar #casamento#conquistarmulher #conquistarumamor #fabricioacunha #fabricioacunhamentor #machomacho#amor #cortespodcast #mulher #homem'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'HOMENS NUNCA PERDOEM UM MULHER TRAÍDORA! #memes #humor #amelhorredpillvemdelas seja bem vindo a mais um vídeo do canal Não se esqueça inscrever no CANAL e deixar o like  além disso ative o sininho de notificações para não perder nenhum vídeo.HOMENS NUNCA PERDOEM UM MULHER TRAÍDORA!'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'MULHER DIZ PARA ELAS NÃO ACEITAREM SER MARMITAS DOS HOMENS! Olá! seja bem vindo a mais um video do canal, hoje vamos falar, sobre  MULHER DIZ PARA ELAS NÃO ACEITAREM SER MARMITAS DOS HOMENS!não se esqueça de se inscrever no canal e deixar o like  além disso ative o sininho de notificações para não perder nenhum video. vamos começar;'"},
                {'role': 'assistant', 'content': "1"},
            ],
            'amostra_2': [
                {'role': 'user', 'content': "Text: 'E vocês, teriam um relacionamento com uma MSol? '"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Ela Era Conhecida Como MARMITA DE MC Mas Agora Resolveu Mudar de Vida Hoje no hackeando a matrix tem uma pessoa reclamando que gostaria de sair pro carnaval, porém ela não tem com quem deixar seu filho. Depois tem a história da pessoa que tinha o apelido de marmita de mc, mas resolveu mudar de vida     ⭐Seja membro para ter acesso a vídeos exclusívos por apenas $2,99 🤏     https://www.youtube.com/channel/UCDrGZXd8k06ifz0-gccIn0w/join#hackeandoamatrix #morenotatuado #homemdealtovalor-----------------------------------------------------------------------------------------------------------Esse é um canal de opinião, onde analiso dinâmicas sociais entre homens e mulheres do ponto de vista de um homem.Meu compromisso é dar minha contribuição sincera e respeitosa sobre assuntos da atualidade que interessam ao homem de alto valor.Quando o homem começa a seguir o próprio caminho aprende que precisa se manter bem informado.💊DEIXE O LIKE!                       💊SE INSCREVA!             👉Compartilhe esse vídeo com alguém que vc gosta:Ela Era Conhecida Como MARMITA DE MC Mas Agora Resolveu Mudar de Vidahttps://youtu.be/2bGr-MrWeCU'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'MULHER EXPLICANDO O QUER DIZER QUANDO AS MULHERES FALAM QUE OS HOMENS SÃO FROUXOS! seja bem vindo a mais um vídeo do canal, hoje vamosMULHER EXPLICANDO O QUER DIZER QUANDO AS MULHERES FALAM QUE OS HOMENS SÃO FROUXOS! !  não se esqueça de se inscrever no CANAL e deixar o like  além disso ative o sininho de notificações para não perder nenhum vídeo. vamos começar'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'VEJA COMO AS MULHERES SÓ LIGAM PARA AS APARÊNCIAS! #humor #memes #amelhorredpillvemdelas #comedia seja bem vindo a mais um vídeo do canal Não se esqueça inscrever no CANAL e deixar o like  além disso ative o sininho de notificações para não perder nenhum vídeo.VEJA COMO AS MULHERES SÓ LIGAM PARA AS APARÊNCIAS!'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: '“Será que vale a pena se apaixonar ou será que vale a pena ficar sozinho?” 🍷🗿 📕 - Antiotario🔱 - Código Masculino🚩 - Red Flags🔞 - Antiporn*🧘 - Lições Perpétuas🔐 - Entrada no Grupo PrivadoEsse é apenas o começo! 📚🗿👇🔗 | https://hotm.art/ComboManual'"},
                {'role': 'assistant', 'content': "0"},
            ],
            'amostra_3': [
                {'role': 'user', 'content': "Text: 'A TRAIÇÃO FEMININA NÃO PODE SER ESQUECIDA! NEM PELOS HOMENS DESPERTOS E NEM PELOS BOIS DOMADOS Há muito tempo atrás já fiz mais de um video falando sobre a questão de que as mulheres já estavam traindo mais do que os homens. Isso faz parte do profano feminino, exacerbado pelo feminismo e que não pode se perder na memória. A inistência em temas como a traição feminina deve ser sempre relembrada para que os homens despertos não venham a esquecer e também serve como mais um exemplo para os críticos raivosos que jogam a culpa somente nos homens pelo decréscimo nos relacionamentos.Logicamente, esse monte de bois de canga não vão rebater o que eu digo. O que eles só sabem fazer é criticar os homens (sabem que não dá em nada) e deixar de fora as maiores culpadas pela derrocada dos casamentos que são as mulheres.Não importa se o feminismo abriu as portas do profano feminino, o que importa é que as mulheres abraçaram essa nova modalidade de comportamento e isso precisa ser jogado na cara desses guampas secas.  https://deolhonacidade.net/detetive-em-feira-relata-que-as-mulheres-traem-mais-que-os-homens/#relacionamento #casamento #crianças'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Você conhece alguma fura-olho aí? | Thiago Schutz #shorts #relacionamento #amor #reflexão #amizade'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'A ROUPA NÃO DEFINE O CARÁTER MAS O CARÁTER DEFINE A ROUPA  #homemdealtovalor #podcast #seducao 📕 Descubra o poder do Método Alpha: método definitivo para transformar sua vida amorosa. Acesse o link abaixo:https://bit.ly/47tJ0bQ- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - Instagram:   Tik Tok:  - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #desenvolvimentomasculino #seducao #homemdealtovalor #relacionamento'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'O que um Homem espera de uma Mulher | Qual a sua opinião? #rafaelaires #antiotario #redpillbrasil🔥 Pare de ser feito de Otário! 🔥📕 Manual AntiOtário✅ Evitar manipulações e jogos emocionais✅ Se posicionar com inteligência nos relacionamentos✅ Torne-se a melhor versão de si mesmo📚 Adquira o seu agora! ⤵➡️| hotm.art/geracaoantiotario🚀 O livro que te ensina a se impor, evitar ciladas e nunca mais ser feito de trouxa.'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Amg casou e a mlher dele no dia quis me d , conto pra ele? #rafaelaires #antiotario #redpill '"},
                {'role': 'assistant', 'content': "0"},
            ],
            'amostra_4': [
                {'role': 'user', 'content': "Text: 'Andando de mãos dadas com a mulher🤫'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'HOMENS, NÃO APELEM PARA O MENOS PIOR! A HONRADINHA DE IGREJA NÃO SERÁ A SALVAÇÃO DE VOCÊS! Um relato horripilante de um conhecido de São Paulo sobre o papo que ele teve com dois amigos.Ambos resolveram apelar para a igreja para arrumar uma esposa menos pior! Eles sabem um pouco da realidade brutal do mundo, mas cometem um erro destrutivo: o de achar que dentro da igreja os possíveis danos de um casamento e divórcio serão diminuídos de forma que vale a pena arriscar.Hoje em dia a meta é não se envolver, pois os homens não possuem seguraça jurídica nenhuma. Fingir que de dentror da igreja sairá uma esposa honrada é arriscar passar por um campo minado. E isso vai além das leis. Homem nenhum sabe no que se tranformará uma pacata mulher de igreja dentro de pouco tempo.Hoje, não importa a procedência, o feminismo adentrou em todas as esferas e contaminou praticamente todas as mulheres. Diminuir os riscos em quantos por cento? Nem eles sabem. Estão no chute. #relacionamento #casamento #igreja'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: '“Estou traindo meu parceiro a anos. Devo confessar?”'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'ELAS EXIGEM TUDO MAS TEM POUCO A OFERECER! Olá! seja bem vindo a mais um vídeo do canal, hoje vamos falar, sobre o porquê que o homem deve malhar e treinar. não se esqueça de se inscrever no canal e deixar o like  além disso ative o sininho de notificações para não perder nenhum vídeo. vamos começar;'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'O SÉKSU COMO PODER DE BARGANHA! COACH DE RELACIONAMENTO APOIA GREVE DE SÉKSU NO CASAMENTO! No meu terceiro vídeo sobre a fata de atração sexual em relação aos homens por parte das mulheres e o consequente uso do séksu como poder de barganha e dominação, termino por comprovar que a atração sexual das mulheres é ditada conforme as conveniências.Nesse vídeo analiso ainda a fala de uma coach de relacionamentos que, como sempre, fala coisas absurdas, entre elas o usso do séksu como ferramenta de greve contra companheiros e maridos.https://youtu.be/iG_w813Ses4?si=6P9TiRajuPYvzMRn#relacionamento #casamento'"},
                {'role': 'assistant', 'content': "1"},
            ],
            'amostra_5': [
                {'role': 'user', 'content': "Text: 'Mulheres sabem como enganar os homens… e muitos nem percebem! Muitos homens entram em relacionamentos sem perceber os truques e manipulações que algumas mulheres usam para conseguir o que querem. Elas sabem como agir para manter o controle e fazer você acreditar que está no comando. No vídeo de hoje, vamos revelar as estratégias femininas e como os homens podem se proteger emocionalmente e financeiramente. Você já passou por algo assim? Comente abaixo! #Relacionamento #Masculinidade #ManipulaçãoFeminina #FabrícioAcunha #AlertaHomens'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'COMO ME PREPARAR PARA AMAR E SER AMADA ✅COMO ME PREPARAR PARA AMAR E SER AMADANeste vídeo eu vou te explicar: como me preparar para amar e ser amada. Você também vai aprender mais sobre amor, amar, e ser amada. Além disso também você vai saber se esta pronta  para amar.🔴GUIA DA MULHER AUTOCONFIANTE🔴https://karinefloriana.com.br/🔴COMO RECONQUISTAR O EX🔴https://pay.kiwify.com.br/B5bvtUE🔴E-BOOK: GUIA PRÁTICO PARA AUMENTAR O AMOR-PRÓPRIO E MELHORAR A AUTOESTIMAhttps://pay.kiwify.com.br/SIiljFSINSCREVA-SE NO NOSSO CANAL PARA RECEBER MAIS CONSELHOS RELACIONADOS A RELACIONAMENTOS AMOROSO.🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🔴Compartilhe esse vídeo : ➤COMO ME PREPARAR PARA AMAR E SER AMADAhttps://youtu.be/Vj_y7N2BCQk🚨Videos que também podem te ajudar🚨➤COMO DESCOBRIR SE ELE TE QUER DE VERDADEhttps://youtu.be/KgiHeongySk➤COMO PARAR DE PENSAR NELE! 4 DICAS PARA MUDAR O PENSAMENTO E ESQUECER ELE.https://youtu.be/67d0yYyVIp8➤QUAIS SÃO AS CINCO PRINCIPAIS CARACTERÍSTICAS DE UM RELACIONAMENTO TÓXICOhttps://youtu.be/s4mve-Svf58➤3 ATITUDES QUE TODO HOMEM PROCURA EM UMA MULHER. SEJA DIFERENTE E IRRESISTÍVEL.https://youtu.be/4ldhbs-sq2E➤4 ATITUDES QUE AFASTAM OS HOMENShttps://youtu.be/sXr7rPLVb5Y➤4 COISAS QUE OS HOMENS ACHAM BONITO EM UMA MULHERhttps://youtu.be/2oARE1DH5Bw➤4 DICAS INFALÍVEIS DE COMO FAZER ELE SENTIR SUA FALTAhttps://youtu.be/p0Tp5mIxfsg➤SERÁ QUE ELE ME QUER? O HOMEM QUE TE QUER VAI FAZER ISTO.https://youtu.be/DzyypBZ0e6A➤3 COISAS QUE OS HOMENS AMAM NAS MULHERES. FAÇA ELE TE QUERERhttps://youtu.be/rMdqdIt8B6o➤GATILHOS VISUAIS QUE DEIXAM O HOMEM LOUCO, A ISCA PERFEITA.https://youtu.be/TUuBavtdsM0===========================================================🔴 #amor #amar #relacionamento #comoconquistaralguem #namoro #comoconquistarumhomem 🔻Me Encontre nas Redes Sociais🔻📸Instagram➤https://www.instagram.com/karinefloriana/📸Tiktok➤http://www.tiktok.com/ 🎬YouTube➤https://www.youtube.com/ 📸Linkedin➤www.linkedin.com/in/karinefloriana📸KWAI➤ https://kwai-video.com/u/ /n8XhvCfK'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'A mãe que perdeu a guarda da filha 1- Adquira o Antiotário 📕2- Domine as lições do nosso livro 3- Faça parte do nosso grupo privado.➡️ https://manualantiotario.com/AntiOtário, livro de desenvolvimento pessoal mais vendido do país. 🫡'"},
                {'role': 'assistant', 'content': "0"},
                {'role': 'user', 'content': "Text: 'MULHER NÃO DA MORAL PARA HOMEM BONZINHO #relacionamento #seducao #homemdealtovalor 👑 Descubra o poder do Método Alpha: método definitivo para transformar sua vida amorosa.  ⚜️ Aprenda os segredos da conquista, desenvolva sua confiança e se torne o homem que as mulheres desejam. ✅️ Com dicas práticas e estratégias comprovadas, e um passo a passo para se tornar o verdadeiro protagonista de sua jornada amorosa. 🔥 Liberte o seu potencial e conquiste o coração das mulheres com o Método Alpha. Chegou a hora de se destacar! Link: https://bit.ly/47tJ0bQ Instagram:   Tik Tok:   #desenvolvimentomasculino #seducao #homemdealtovalor #relacionamento'"},
                {'role': 'assistant', 'content': "1"},
                {'role': 'user', 'content': "Text: 'Mulher e💲ao mesmo tempo💰😍'"},
                {'role': 'assistant', 'content': "1"},
            ],
        },
    },
}

In [11]:
# Para rodar para todos os zero shot de 1 só vez
import pandas as pd
import logging
from tqdm import tqdm
import transformers
import os

# --- Configurações Iniciais ---
CHUNK_SIZE = 404
transformers.logging.set_verbosity_error()
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', filename='processamento_llm.log', filemode='w')


all_tests = [
    # --- Teste 1: Zero-Shot, Português, Sem Conceito ---
    {
        "name": "teste1_pt_sem_conceito",
        "system_prompt_base": (
            "Você é um classificador de texto altamente preciso de títulos e descrições do Youtube. "
            "Sua única função é analisar o texto e responder estritamente com '0' ou '1'.\n"
            "A pergunta que você deve responder é: O texto é {task_definition}?"
            "1: O texto É {task_definition}\n"
            "0: O texto NÃO é {task_definition}\n"
            "Não forneça nenhuma explicação, apenas o número."
        ),
        "task_definitions": {
            "toxicidade": "TÓXICO",
            "misoginia": "MISÓGINO"
        },
        "few_shot_config": {
            "toxicidade": {"output_prefix": "results_toxic"},
            "misoginia": {"output_prefix": "results_mysogyny"}
        }
    },

    # --- Teste 2: Zero-Shot, Inglês, Sem Conceito ---
    {
        "name": "teste2_en_sem_conceito",
        "system_prompt_base": (
            "You are a highly accurate text classifier for Youtube titles and descriptions. "
            "Your sole function is to analyze the text and respond strictly with '0' or '1'.\n"
            "The question you must answer is: Is the text {task_definition}?\n"
            "1: The text IS {task_definition}\n"
            "0: The text is NOT {task_definition}\n"
            "Do not provide any explanation, just the number."
        ),
        "task_definitions": {
            "toxicidade": "TOXIC",
            "misoginia": "MISOGYNISTIC"
        },
        "few_shot_config": {
            "toxicidade": {"output_prefix": "results_toxic"},
            "misoginia": {"output_prefix": "results_mysogyny"}
        }
    },

    # --- Teste 3: Zero-Shot, Português, Com Conceito ---
    {
        "name": "teste3_pt_com_conceito",
        "system_prompt_base": (
            "Você é um classificador de texto altamente preciso de títulos e descrições do Youtube. "
            "Sua única função é analisar o texto e responder estritamente com '0' ou '1'.\n"
            "Você deve analisar se o texto contém {task_definition}. Assim, responda:"
            "1: Se o texto se encaixa em qualquer uma das categorias de conceito listadas \n"
            "0: Se o texto NÃO se encaixa em qualquer uma das categorias de conceito listadas\n"
            "Não forneça nenhuma explicação, apenas o número."
        ),
        "task_definitions": {
            "toxicidade": "TOXICIDADE - Utilize as seguintes definições para guiar sua classificação: \n1. Ódio e Assédio:\n1.1 Doxing: Divulgação de informações pessoais de um alvo para o público em geral, com o objetivo de prejudicá-lo.\n1.2 Ataque à identidade: Consiste em ataques severos a indivíduos ou grupos com base em sua afiliação a um grupo protegido ou vulnerável.\n1.3 Falsa representação de identidade: É a disseminação de declarações que comunicam deturpações pejorativas, estereótipos e outras generalizações insultuosas sobre populações protegidas ou vulneráveis.\n1.4 Insulto: Ataque pessoal direcionado, como xingamentos, comentários negativos, depreciativos e inflamatórios sobre o indivíduo.\n1.5 Agressão Sexual: Inclui avanços sexuais indesejados, sexualização indesejável e compartilhamento não consensual de conteúdo sexual, entre outras formas de conversas sexuais não solicitadas.\n1.6 Ameaça de Violência: Refere-se a conteúdo que expressa o desejo de matar ou infligir dano físico, celebra atos violentos ou incita outros a cometerem violência.\n2. Dano autoinfligido:\n2.1 Promoção de transtorno alimentar: Abrange conteúdo que promove transtornos alimentares como escolhas de estilo de vida legítimas ou glorifica corpos excessivamente magros.\n2.2 Autolesão: Ideação de se machucar fisicamente de propósito para lidar com o sofrimento emocional, incluindo também a ideação suicida.\n3. Dano ideológico:\n3.1 Extremismo, terrorismo e crime organizado: Inclui conteúdo relacionado ao recrutamento, elogio ou assistência a grupos extremistas, terroristas ou de crime organizado.\n3.2 Desinformação: É a informação falsa ou enganosa, que pode ser disseminada com ou sem a intenção deliberada de prejudicar.\n4. Exploração:\n4.1 Serviços sexuais adultos: Refere-se à promoção ou solicitação de serviços sexuais ilegais que exploram participantes frequentemente vulneráveis.\n4.2 Material de abuso sexual infantil: É definido como conteúdo que envolve o abuso e a exploração sexual de qualquer pessoa menor de dezoito anos.\n4.3 Golpes: São tentativas de enganar uma pessoa para que ela forneça fundos ou informações sensíveis usando técnicas fraudulentas ou invasivas.\n",
            "misoginia": "MISOGINIA - Utilize as seguintes definições para guiar sua classificação: \n1. Ameaças, planos para prejudicar e incitamento:\n1.1 Ameaças de danos: Expressar desejo de prejudicar física, sexual ou emocionalmente uma mulher ou grupo de mulheres.\n1.2 Incitamento e incentivo ao dano: Encorajar ou justificar que outros prejudiquem uma mulher ou grupo de mulheres.\n2. Depreciação:\n2.1 Ataques descritivos: Fazer generalizações negativas sobre as habilidades, aparência, intelecto ou caráter das mulheres.\n2.2 Ataques agressivos e emotivos: Expressar ódio ou nojo contra as mulheres, usando insultos de gênero ou acusações infundadas.\n2.3 Ataques desumanizantes e objetificação sexual explícita: Comparar mulheres a animais/objetos ou reduzi-las a objetos sexuais.\n3. Animosidade:\n3.1 Uso casual de gírias de gênero, profanidades e insultos: Usar termos pejorativos para mulheres de forma casual, não como um ataque direto.\n3.2 Diferenças imutáveis de gênero e estereótipos de gênero: Afirmar diferenças 'naturais' entre homens e mulheres para implicar inferioridade ou limitar papéis (inclui a maioria das piadas sexistas).\n3.3 Elogios de gênero ambíguos: Elogiar mulheres de forma que reforce estereótipos de fragilidade, ou que reduza seu valor à sua atratividade.\n3.4 Explicações condescendentes ou conselhos indesejados: Oferecer conselhos paternalistas a mulheres sobre assuntos que elas dominam ('mansplaining').\n4. Discussão Preconceituosa:\n4.1 Apoio ao mau tratamento de mulheres individuais: Justificar, negar ou subestimar o mau tratamento de uma mulher.\n4.2 Apoio à discriminação sistêmica contra mulheres como grupo: Justificar, negar ou subestimar a discriminação contra as mulheres como um grupo.\n"
        },
        "few_shot_config": {
            "toxicidade": {"output_prefix": "results_toxic"},
            "misoginia": {"output_prefix": "results_mysogyny"}
        }
    },

    # --- Teste 4: Zero-Shot, Inglês, Com Conceito ---
    {
        "name": "teste4_en_com_conceito",
        "system_prompt_base": (
            "You are a highly accurate text classifier for Youtube titles and descriptions. "
            "Your sole function is to analyze the text and respond strictly with '0' or '1'.\n"
            "You must analyze if the text contains {task_definition}. Accordingly, respond:\n"
            "1: If the text fits into any of the listed concept categories\n"
            "0: If the text does NOT fit into any of the listed concept categories\n"
            "Do not provide any explanation, just the number."
        ),
        "task_definitions": {
            "toxicidade": "TOXICITY - Use the following definitions to guide your classification: \n1. Hate and Harassment:\n1.1 Doxing: Disclosure of a target's personal information to the general public with the intent to harm them.\n1.2 Identity Attack: Consists of severe attacks on individuals or groups based on their affiliation with a protected or vulnerable group.\n1.3 Misrepresentation of Identity: Is the dissemination of statements that communicate pejorative misrepresentations, stereotypes, and other insulting generalizations about protected or vulnerable populations.\n1.4 Insult: A directed personal attack, such as name-calling, negative, derogatory, and inflammatory comments about the individual.\n1.5 Sexual Aggression: Includes unwanted sexual advances, unwelcome sexualization, and non-consensual sharing of sexual content, among other forms of unsolicited sexual conversations.\n1.6 Threat of Violence: Refers to content that expresses the desire to kill or inflict physical harm, celebrates violent acts, or incites others to commit violence.\n2. Self-Harm:\n2.1 Promotion of Eating Disorders: Covers content that promotes eating disorders as legitimate lifestyle choices or glorifies excessively thin bodies.\n2.2 Self-Injury: Ideation of purposely physically hurting oneself to cope with emotional distress, including suicidal ideation.\n3. Ideological Harm:\n3.1 Extremism, Terrorism, and Organized Crime: Includes content related to the recruitment, praise, or assistance of extremist, terrorist, or organized crime groups.\n3.2 Disinformation: Is false or misleading information, which can be disseminated with or without the deliberate intention to cause harm.\n4. Exploitation:\n4.1 Adult Sexual Services: Refers to the promotion or solicitation of illegal sexual services that exploit often-vulnerable participants.\n4.2 Child Sexual Abuse Material: Is defined as content that involves the sexual abuse and exploitation of any person under the age of eighteen.\n4.3 Scams: Are attempts to deceive a person into providing funds or sensitive information using fraudulent or invasive techniques.\n",
            "misoginia": "MISOGYNY - Use the following definitions to guide your classification: \n1. Threats, plans to harm, and incitement:\n1.1 Threats of harm: Expressing a desire to physically, sexually, or emotionally harm a woman or group of women.\n1.2 Incitement and encouragement of harm: Encouraging or justifying others to harm a woman or group of women.\n2. Derogation:\n2.1 Descriptive attacks: Making negative generalizations about women's skills, appearance, intellect, or character.\n2.2 Aggressive and emotive attacks: Expressing hatred or disgust against women, using gendered slurs or baseless accusations.\n2.3 Dehumanizing attacks and explicit sexual objectification: Comparing women to animals/objects or reducing them to sexual objects.\n3. Animosity:\n3.1 Casual use of gendered slurs, profanity, and insults: Using derogatory terms for women casually, not as a direct attack.\n3.2 Immutable gender differences and gender stereotypes: Asserting 'natural' differences between men and women to imply inferiority or limit roles (includes most sexist jokes).\n3.3 Ambiguous gendered praise: Praising women in a way that reinforces stereotypes of fragility, or that reduces their value to their attractiveness.\n3.4 Condescending explanations or unsolicited advice: Offering paternalistic advice to women on subjects they are knowledgeable about ('mansplaining').\n4. Prejudiced Discussion:\n4.1 Support for the mistreatment of individual women: Justifying, denying, or understating the mistreatment of a woman.\n4.2 Support for systemic discrimination against women as a group: Justifying, denying, or understating discrimination against women as a group.\n"
        },
        "few_shot_config": {
            "toxicidade": {"output_prefix": "results_toxic"},
            "misoginia": {"output_prefix": "results_mysogyny"}
        }
    }
]


for test_config in all_tests:
    test_name = test_config["name"]
    SYSTEM_PROMPT_BASE = test_config["system_prompt_base"]
    TASK_DEFINITIONS = test_config["task_definitions"]
    FEW_SHOT_CONFIG = test_config["few_shot_config"]

    logging.info(f"############################################################")
    logging.info(f"##### INICIANDO GRUPO DE TESTE: {test_name} #####")
    logging.info(f"############################################################")

    for fold_number in range(1, 6):
        for task_name_pt, config in FEW_SHOT_CONFIG.items():

            # --- MUDANÇA 1: ADICIONA LOOP PARA AS 5 AMOSTRAS ---
            for sample_number in range(1, 6):

                logging.info(f"============================================================")
                logging.info(f"PROCESSANDO: Teste '{test_name}' - Fold {fold_number} - Tarefa: {task_name_pt} - Amostra: {sample_number}")
                logging.info(f"============================================================")

                # Checa se a combinação de fold/tarefa/amostra existe nos dados
                fold_key = f'fold_{fold_number}'
                sample_key = f'amostra_{sample_number}'
                if not ALL_FEW_SHOT_SAMPLES.get(fold_key, {}).get(task_name_pt, {}).get(sample_key):
                    logging.warning(f"Amostra {sample_number} para Fold {fold_number} ({task_name_pt}) não encontrada. Pulando.")
                    continue

                # --- Lógica original de montagem do prompt (sem mudanças) ---
                task_definition_key = 'misoginia' if task_name_pt == 'misoginia' else 'toxicidade'
                task_definition = TASK_DEFINITIONS.get(task_definition_key, "")
                dynamic_system_prompt = SYSTEM_PROMPT_BASE.format(task_definition=task_definition)

                sufixo_arquivo = ""
                if task_name_pt == "toxicidade":
                    sufixo_arquivo = "_TOXICIDADE"
                elif task_name_pt == "misoginia":
                    sufixo_arquivo = "_MISOGINIA"

                input_filename = f"folds/videos_fold_{fold_number}_estratificado{sufixo_arquivo}.csv"

                if not os.path.exists(input_filename):
                    logging.warning(f"Arquivo {input_filename} não encontrado. Pulando.")
                    continue

                # --- MUDANÇA 2: Reseta os resultados para cada amostra ---
                all_results_for_run = []
                reader = pd.read_csv(input_filename, chunksize=CHUNK_SIZE)

                for chunk_df in tqdm(reader, desc=f"{test_name} - F{fold_number} - S{sample_number} ({task_name_pt})"):
                    if chunk_df.empty:
                        continue

                    batch_messages_text = chunk_df['text'].astype(str).tolist()
                    batch_conversations = []

                    prefix = "Text:" if 'en' in test_name else "Texto:"

                    for msg_text in batch_messages_text:
                        formatted_msg = f"{prefix} '{msg_text}'"

                        conversation = [{"role": "system", "content": dynamic_system_prompt}]

                        # --- MUDANÇA 3: Busca dinâmica dos exemplos corretos ---
                        current_examples = ALL_FEW_SHOT_SAMPLES[fold_key][task_name_pt][sample_key]
                        conversation.extend(current_examples)

                        conversation.append({"role": "user", "content": formatted_msg})
                        batch_conversations.append(conversation)

                    if not batch_conversations:
                        continue

                    # O código do pipeline processa os prompts
                    batch_prompts_formatted = tokenizer.apply_chat_template(
                        batch_conversations, tokenize=False, add_generation_prompt=True
                    )
                    batch_results = pipe(batch_prompts_formatted, **generation_params)

                    results_to_save_this_chunk = []
                    for i, result in enumerate(batch_results):
                        llm_response = result[0]['generated_text'].strip()
                        original_message = batch_messages_text[i]

                        results_to_save_this_chunk.append({
                            "original_id": chunk_df['id_video_anonimizado'].iloc[i],
                            "original_prompt": original_message,
                            "llm_response": llm_response
                        })

                    if results_to_save_this_chunk:
                        all_results_for_run.extend(results_to_save_this_chunk)

                # --- MUDANÇA 4: Bloco de salvamento agora está dentro do loop da amostra ---
                if all_results_for_run:
                    output_prefix = config["output_prefix"]
                    # --- O nome do arquivo agora inclui a amostra ---
                    output_filename = f"resultados_few/{output_prefix}_{test_name}_fold_{fold_number}_amostra_{sample_number}.csv"

                    final_df = pd.DataFrame(all_results_for_run)
                    final_df.to_csv(output_filename, index=False)
                    logging.info(f"Resultados salvos em: {output_filename}")
                else:
                    logging.warning(f"Nenhum resultado gerado para Teste '{test_name}', Fold {fold_number}, Tarefa: {task_name_pt}, Amostra: {sample_number}")

logging.info("============================================================")
logging.info("TODOS OS TESTES FORAM CONCLUÍDOS!")
logging.info("============================================================")

teste1_pt_sem_conceito - F1 - S1 (toxicidade): 1it [01:04, 64.55s/it]
teste1_pt_sem_conceito - F1 - S2 (toxicidade): 1it [01:12, 72.15s/it]
teste1_pt_sem_conceito - F1 - S3 (toxicidade): 1it [01:06, 66.41s/it]
teste1_pt_sem_conceito - F1 - S4 (toxicidade): 1it [01:03, 63.00s/it]
teste1_pt_sem_conceito - F1 - S5 (toxicidade): 1it [01:00, 60.60s/it]
teste1_pt_sem_conceito - F1 - S1 (misoginia): 1it [01:50, 110.10s/it]
teste1_pt_sem_conceito - F1 - S2 (misoginia): 1it [01:21, 81.54s/it]
teste1_pt_sem_conceito - F1 - S3 (misoginia): 1it [00:47, 47.31s/it]
teste1_pt_sem_conceito - F1 - S4 (misoginia): 1it [02:54, 174.44s/it]
teste1_pt_sem_conceito - F1 - S5 (misoginia): 1it [02:06, 126.73s/it]
teste1_pt_sem_conceito - F2 - S1 (toxicidade): 1it [01:23, 83.49s/it]
teste1_pt_sem_conceito - F2 - S2 (toxicidade): 1it [01:12, 72.50s/it]
teste1_pt_sem_conceito - F2 - S3 (toxicidade): 1it [01:31, 91.88s/it]
teste1_pt_sem_conceito - F2 - S4 (toxicidade): 1it [01:06, 66.94s/it]
teste1_pt_sem_conceito